In [1]:
import sys
import array
import os
import timeit
import contextlib
import math
import urllib.request
import zipfile
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
from collections import OrderedDict, Counter, defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
!mkdir -p output # Creación de subdirectorio auxiliar para salidas

Los datos para esta práctica están disponibles como archivo .zip en este [enlace](http://web.stanford.edu/class/cs276/pa/pa3-data.zip). El _dataset_ está dividido en dos conjuntos separados:
1. **Conjunto de entrenamiento** formado por 731 consultas (`pa3.(signal|rel).train`)
2. **Conjunto de test** formado por 124 consultas (`pa3.(signal|rel).dev`)

La idea será que, a la vez que ajustemos y maximicemos el rendimiento en el conjunto de entrenamiento, verifiquemos el rendimiento de los parámetros ajustados en el conjunto de desarrollo para asegurarnos de que no estamos sobreajustando el modelo.

In [3]:
# Descarga del dataset:
data_dir = 'pa3-data'
data_url = 'http://web.stanford.edu/class/cs276/pa/{}.zip'.format(data_dir)
urllib.request.urlretrieve(data_url, '{}.zip'.format(data_dir))

# Descomprimimos el archivo .zip:
with zipfile.ZipFile('{}.zip'.format(data_dir), 'r') as zip_fh:
    zip_fh.extractall()
print('Datos descargados y descomprimidos en {}...\n'.format(data_dir))

# Imprimimos la estructura del directorio:
print('Estructura del directorio:')
print(data_dir + os.path.sep)
for sub_dir in os.listdir(data_dir):
    if not sub_dir.startswith('.'):
        print('  - ' + sub_dir)

Datos descargados y descomprimidos en pa3-data...

Estructura del directorio:
pa3-data/
  - BSBI.dict
  - terms.dict
  - pa3.rel.train
  - pa3.rel.dev
  - pa3.signal.dev
  - docs.dict
  - pa3.signal.train


## Descripción de los archivos

### Archivos de "señal"

- **pa3.signal.(_train_|_test_)**: Contienen el conjunto de consultas correspondiente (_train_|_test_) junto con los documentos devueltos para cada consulta individual por un determinado motor de búsqueda. La lista de documentos se encuentra convenientemente mezclada y no está en el mismo orden que el devuelto por el motor de búsqueda empleado originalmente. Cada consulta tiene siempre 10 o menos documentos de respuesta. Por ejemplo, el formato de un par consulta-documento $(q,d)$ es el siguiente:

In [4]:
# Mostramos un conjunto de líneas ilustrativo del archivo de training (la primera
# consulta ("query:"), con sus primeros dos documentos ("url:") completos (líneas 0 a 29),
# y otro par consulta-documentos adicional más adelante en el fichero (líneas 233
# en adelante):
filename = os.path.join(data_dir, "pa3.signal.train")
with open(filename, 'r', encoding = 'utf8') as f:
    lines = f.readlines()
    for l in lines[0:29]:
        print(l, end="")
    print("    ...\n")
    for l in lines[233:256]:
        print(l, end="")
    print("    ...")

query: stanford aoerc pool hours
  url: http://events.stanford.edu/2014/February/18/
    title: events at stanford tuesday february 18 2014
    header: stanford university event calendar
    header: teaching sex at stanford
    header: rodin the complete stanford collection
    header: stanford rec trx suspension training
    header: memorial church open visiting hours
    header: alternative transportation counseling tm 3 hour stanford univ shc employees retirees family members
    body_hits: stanford 239 271 318 457 615 642 663 960 966 971
    body_hits: aoerc 349 401 432 530 549 578 596
    body_hits: pool 521
    body_length: 981
    pagerank: 1
  url: http://events.stanford.edu/2014/February/6/
    title: events at stanford thursday february 6 2014
    header: stanford university event calendar
    header: stanford woods environmental forum featuring roz naylor
    header: stanford school of earth sciences alumni reception at nape
    header: an evening with stanford alumnus and p

Atendiendo a la salida anterior se ejemplifica perfectamente la estructura de estos archivos de señal. El patrón se repite para cada URL, hasta que todas las URL (siempre 10 o menos) de la correspondiente consulta estén completas. A continuación este patrón general se repite para cada consulta posterior. Hay solo un campo `title`, otro `pagerank`, y otro `body_length` para cada URL, pero puede haber múltiples campos `header`, `body_hits` y `anchor_text` (junto con los correspondientes `stanford_anchor_count`). He aquí el significado de los mencionados campos:

* El campo `title` es el título de la página.

* El campo `pagerank` es un número entero de 0 a 9 que indica una calidad independiente de la consulta de la página (cuanto mayor sea dicho valor, mejor será la calidad de la página).

* El campo `body_length` indica cuántos términos están presentes en el cuerpo del documento.

* Cada campo `header` indica un subtítulo contenido en la página.

* Cada campo `body_hits` especifica, para cada término de la consulta $q$, la _posting list_ posicional para ese término en el documento (posiciones de la palabra en el mismo, siempre ordenadas en orden creciente).

* Cada campo `anchor_text`, siempre seguido inmediatamente por un subcampo `stanford_anchor_count` correspondiente, indica el texto de un enlace desde cualquier página del dominio de páginas web en las que está basado el dataset ([https://www.stanford.edu/](https://www.stanford.edu/)) al documento actual, junto con el número total de enlaces con dicho texto en el dataset. Por ejemplo, si el texto de anclaje (`anchor_text`) es _"Stanford math department"_ y el recuento (`stanford_anchor_count`) es 9, eso significa que hay nueve enlaces a la página actual (desde otras páginas en https://www.stanford.edu/) donde el texto de anclaje es exactamente _"Stanford math department"_. Así, en el ejemplo anterior del segundo documento de la segunda consulta, podemos ver que el anclaje _"Cardinal nights"_ aparece en 208 páginas del dominio https://www.stanford.edu/ apuntando al documento https://alcohol.stanford.edu/cardinal-nights en cuestión.

El campo `pagerank` será empleado más adelante como ejemplo de _feature_ no textual, mientras que el resto de campos se corresponden con distintas _zonas_ del documento, que actuarán por tanto como _features_ textuales diferenciadas.

### Archivos de relevancias

* **pa3.rel.(train|dev)**: Estos archivos contienen una lista de juicios de relevancia (etiquetados manualmente) para cada par consulta-documento $(q,d)$ en los respectivos archivos de señal (_train|test_). El valor de relevancia es siempre un entero entre -1 y 3, con un dato mayor significando que el documento tiene más relevancia. −1 significaría que el documento ha sido simplemente ignorado. De nuevo, el patrón se repite para cada consulta, hasta que se alcanza el final del archivo.

Por ejemplo:

In [5]:
# Mostramos las primeras filas del archivo:
filename = os.path.join(data_dir, "pa3.rel.train")
with open(filename, 'r', encoding = 'utf8') as f:
    lines = f.readlines()
    for l in lines[0:25]:
        print(l.strip())
print("...")

query: stanford aoerc pool hours
url: http://events.stanford.edu/2014/February/18/ 0.0
url: http://events.stanford.edu/2014/February/6/ 0.0
url: http://events.stanford.edu/2014/March/13/ 0.0
url: http://events.stanford.edu/2014/March/3/ 0.0
url: http://med.stanford.edu/content/dam/sm/hip/documents/FreeFitnessWeek.pdf 0.0
url: http://web.stanford.edu/group/masters/pool.html 1.0
url: https://alumni.stanford.edu/get/page/perks/PoolAndGyms 1.5
url: https://cardinalrec.stanford.edu/facilities/aoerc/ 2.0
url: https://explorecourses.stanford.edu/search?view=catalog&filter-coursestatus-Active=on&page=0&catalog=&q=PE+128%3A+Swimming%3A+Beginning+I&collapse= 0.5
url: https://glo.stanford.edu/events/stanford-rec-open-house 0.5
query: alumni association benefits
url: http://alumni.stanford.edu/get/page/membership/benefits/creditcard 2.0
url: http://alumni.stanford.edu/get/page/membership/benefits/libraries 2.0
url: http://alumni.stanford.edu/get/page/membership/students 2.0
url: https://alumni-esc

Finalmente, las funciones de _ranking_ requieren también ciertos estadísticos a nivel de colección de documento (tales como la frecuencia inversa de documento, o _idf_), que no están contenidos en los anteriores archivos. Para eso proporcionamos los archivos **docs.dict**, **terms.dict** y **BSBI.dict**, con los que pueden calcularse los correspondientes valores de _idf_ necesarios. En este caso, se trata simplemente de archivos binarios (`pickle` de python):

## _Parsing_ de los archivos del _dataset_

En primer lugar, definimos las clases `Query` y `Document` y la función `load_train_data`, necesarias para hacer el _parsing_ de los datos textuales contenidos en los ficheros de entrada:

### Clase _Query_

In [6]:
class Query:
    """Clase utilizada para almacenar una consulta."""
    def __init__(self, query):
        self.query_words = query.split(" ")

    def __iter__(self):
        for w in self.query_words:
            yield w

    def __eq__(self, other):
        if not isinstance(other, Query):
            return False
        return self.query_words == other.query_words

    def __hash__(self):
        return hash(str(self))

    def __str__(self):
        return " ".join(self.query_words)

    __repr__ = __str__

### Clase _Document_

In [7]:
class Document:
    """Clase utilizada para almacenar información útil para un documento."""
    def __init__(self, url):
        self.url = url        # Cadena
        self.title = None     # Cadena
        self.headers = None   # [Lista de cadenas]
        self.body_hits = None # Diccionario: Término->[Lista de posiciones]
        self.body_length = 0  # Entero
        self.pagerank = 0     # Entero
        self.anchors = None   # Diccionario: Cadena->[Conteo total de ocurrencias (anchor_counts)]

    def __iter__(self):
        for u in self.url:
            yield u

    def __str__(self):
        result = [];
        NEW_LINE = "\n"
        result.append("url: "+ self.url + NEW_LINE);
        if (self.title is not None): result.append("title: " + self.title + NEW_LINE);
        if (self.headers is not None): result.append("headers: " + str(self.headers) + NEW_LINE);
        if (self.body_hits is not None): result.append("body_hits: " + str(self.body_hits) + NEW_LINE);
        if (self.body_length != 0): result.append("body_length: " + str(self.body_length) + NEW_LINE);
        if (self.pagerank != 0): result.append("pagerank: " + str(self.pagerank) + NEW_LINE);
        if (self.anchors is not None): result.append("anchors: " + str(self.anchors) + NEW_LINE);
        return " ".join(result)

    __repr__ = __str__

### Función _load_train_data_

La siguiente función realiza el _parsing_ de toda la información textual relevante contenida en un archivo de _training | test_ dado, ya en una estructura python (`query_dict`) fácilmente accesible programáticamente:

In [8]:
def load_train_data(feature_file_name):
    """
    Args:
        feature_file_name: Camino al fichero con las features de entrada.

    Returns:
       query_dict: Diccionario de tipo "Consulta -> (URL -> Documento)"". Por ejemplo:
        {computer science master: {'http://cs.stanford.edu/people/eroberts/mscsed/Admissions-MSInCSEducation.html':
          {title: ms in computer science education stanford computer science
           headers: ["master's degree in computer science education"]
           body_hits: {'computer': [15], 'science': [16]}
           body_length: 741
           anchors: {'computer science': 2},
          'http://scpd.stanford.edu/online-engineering-courses.jsp': title: online engineering courses stanford university
           headers: ['computer science and information technology']
           body_hits: {'science': [136], 'master': [188], 'computer': [223]}
           body_length: 687,
           }
         ...,
         }
    """
    line = None
    url = None
    anchor_text = None
    query = None
    query_dict = {}
    try:
        with open(feature_file_name, 'r', encoding = 'utf8') as f:
            for line in f:
                token_index = line.index(":")
                key = line[:token_index].strip()
                value = line[token_index + 1:].strip()
                if key == "query":
                    # Nueva consulta:
                    query = Query(value)
                    query_dict[query] = {}
                elif key == "url":
                    # Nuevo documento encontrado para la actual consulta:
                    url = value;
                    query_dict[query][url] = Document(url);
                elif key == "title":
                    query_dict[query][url].title = str(value);
                elif key == "header":
                    if query_dict[query][url].headers is None:
                        query_dict[query][url].headers = []
                    query_dict[query][url].headers.append(value)
                elif key == "body_hits":
                    if query_dict[query][url].body_hits is None:
                        query_dict[query][url].body_hits = {}
                    temp = value.split(" ",maxsplit=1);
                    term = temp[0].strip();
                    if term not in query_dict[query][url].body_hits:
                        positions_int = []
                        query_dict[query][url].body_hits[term] = positions_int
                    else:
                        positions_int = query_dict[query][url].body_hits[term]
                    positions = temp[1].strip().split(" ")
                    for position in positions:
                        positions_int.append(int(position))
                elif key == "body_length":
                    query_dict[query][url].body_length = int(value);
                elif key == "pagerank":
                    query_dict[query][url].pagerank = int(value);
                elif key == "anchor_text":
                    anchor_text = value
                    if query_dict[query][url].anchors is None:
                        query_dict[query][url].anchors = {}
                elif key == "stanford_anchor_count":
                    query_dict[query][url].anchors[anchor_text] = int(value)
    except FileNotFoundError:
        print(f"Fichero {feature_file_name} no encontrado!")

    return query_dict

# Cargamos archivo completo de training:
file_name = os.path.join(data_dir, "pa3.signal.train")
query_dict = load_train_data(file_name)

In [10]:
# Acceder a todos los documentos (dados por sus URLs) de una consulta:
query_text = "stanford aoerc pool hours"
#### COMPLETAR. Asigna el valor adecuado a la variable: 
query = query_dict[Query(query=query_text)]
#### FIN COMPLETAR
print(f'Documentos (URLs) para la consulta "{query_text}":')
for url in query.keys():
    print(f"  {url}")

# Acceder a un documento completo, dado por su URL, dentro de la consulta:
query_text = "stanford aoerc pool hours"
url_text = "http://events.stanford.edu/2014/February/18/"
print(f'\nDocumento "{url_text}" correspondiente a la consulta "{query_text}":')
#### COMPLETAR: Imprimir el documento solicitado
print(query_dict[Query(query_text)][url_text])
#### FIN COMPLETAR

# Acceder sólo a un campo específico dentro de un documento:
print(f'Campo "body_hits" para documento "{url_text} correspondiente a la consulta "{query_text}:"')
#### COMPLETAR: Imprimir el campo solicitado

Documentos (URLs) para la consulta "stanford aoerc pool hours":
  http://events.stanford.edu/2014/February/18/
  http://events.stanford.edu/2014/February/6/
  http://events.stanford.edu/2014/March/13/
  http://events.stanford.edu/2014/March/3/
  http://med.stanford.edu/content/dam/sm/hip/documents/FreeFitnessWeek.pdf
  http://web.stanford.edu/group/masters/pool.html
  https://alumni.stanford.edu/get/page/perks/PoolAndGyms
  https://cardinalrec.stanford.edu/facilities/aoerc/
  https://explorecourses.stanford.edu/search?view=catalog&filter-coursestatus-Active=on&page=0&catalog=&q=PE+128%3A+Swimming%3A+Beginning+I&collapse=
  https://glo.stanford.edu/events/stanford-rec-open-house

Documento "http://events.stanford.edu/2014/February/18/" correspondiente a la consulta "stanford aoerc pool hours":
url: http://events.stanford.edu/2014/February/18/
 title: events at stanford tuesday february 18 2014
 headers: ['stanford university event calendar', 'teaching sex at stanford', 'rodin the comple

Documentos (URLs) para la consulta "stanford aoerc pool hours":
  http://events.stanford.edu/2014/February/18/
  http://events.stanford.edu/2014/February/6/
  http://events.stanford.edu/2014/March/13/
  http://events.stanford.edu/2014/March/3/
  http://med.stanford.edu/content/dam/sm/hip/documents/FreeFitnessWeek.pdf
  http://web.stanford.edu/group/masters/pool.html
  https://alumni.stanford.edu/get/page/perks/PoolAndGyms
  https://cardinalrec.stanford.edu/facilities/aoerc/
  https://explorecourses.stanford.edu/search?view=catalog&filter-coursestatus-Active=on&page=0&catalog=&q=PE+128%3A+Swimming%3A+Beginning+I&collapse=
  https://glo.stanford.edu/events/stanford-rec-open-house

Documento "http://events.stanford.edu/2014/February/18/" correspondiente a la consulta "stanford aoerc pool hours":
url: http://events.stanford.edu/2014/February/18/
 title: events at stanford tuesday february 18 2014
 headers: ['stanford university event calendar', 'teaching sex at stanford', 'rodin the complete stanford collection', 'stanford rec trx suspension training', 'memorial church open visiting hours', 'alternative transportation counseling tm 3 hour stanford univ shc employees retirees family members']
 body_hits: {'stanford': [239, 271, 318, 457, 615, 642, 663, 960, 966, 971], 'aoerc': [349, 401, 432, 530, 549, 578, 596], 'pool': [521]}
 body_length: 981
 pagerank: 1

Campo "body_hits" para documento "http://events.stanford.edu/2014/February/18/ correspondiente a la consulta "stanford aoerc pool hours:"
{'stanford': [239, 271, 318, 457, 615, 642, 663, 960, 966, 971], 'aoerc': [349, 401, 432, 530, 549, 578, 596], 'pool': [521]}

## Construcción del diccionario IDF

Construiremos ahora la clase que nos permitirá computar los IDF de los términos a partir de los correspondientes archivos <b>.dict</b> que usaremos también como entrada (similares a los utilizados en el _notebook_ de la práctica 1).


### Clase _IdMap_

Comenzamos creando la clase IdMap, para mapear cadenas (_tokens_) a sus correspondientes identificadores numéricos (_tokenIDs_) y viceversa. Valdrá tanto para documentos (identificados por su URL) como para términos (cadenas con un simple token):

In [11]:
class IdMap:
    """Clase auxiliar para almacenar mapeos entre strings e identificadores numéricos de tokens."""
    def __init__(self):
        self.str_to_id = {}
        self.id_to_str = []

    def __len__(self):
        """Devuelve el número de términos almacenados en el IdMap"""
        return len(self.id_to_str)

    def _get_str(self, i):
        """Devuelve la cadena correspondiente a un determinado identificador (`i`)."""
        return self.id_to_str[i]

    def _get_id(self, s):
        """Devuelve el identificador (id) correspondiente a una cadena (`s`).
        En el caso de que `s` no esté aún en el IdMap, asigna un nuevo identificador y devuelve el nuevo id creado.
        """
        if s not in self.str_to_id:
            self.str_to_id[s] = len(self.id_to_str)
            self.id_to_str.append(s)
        return self.str_to_id[s]

    def __getitem__(self, key):
        """Acceso a un elemento por identificador o por cadena de texto.
           Si `key` es un entero, usa el método _get_str;
           Si `key` es una cadena, usa el método _get_id;"""
        if type(key) is int:
            return self._get_str(key)
        elif type(key) is str:
            return self._get_id(key)
        else:
            raise TypeError

### Lectura de diccionarios (_docs_, _terms_, _postings\_dict_)

Cargamos ahora los archivos `docs.dict`, `terms.dict` y `BSBI.dict` creados en la práctica 1, e imprimimos sus respectivas longitudes y los primeros registros para recordar sus respectivas estructuras:

In [12]:
# Leemos los tres archivos e imprimimos sus longitudes:
with open("pa3-data/terms.dict", 'rb') as f:
    terms = pkl.load(f)
with open("pa3-data/docs.dict", 'rb') as f:
    docs = pkl.load(f)
with open('pa3-data/BSBI.dict', 'rb') as f:
    postings_dict, termsID = pkl.load(f)
print(f"Leídos {len(docs)} documentos y {len(terms)} términos")

# Chequeamos la consistencia de longitudes:
assert len(terms) == len(termsID) == len(postings_dict), \
       "Inconsistencia en longitudes de terms, termsID y/o postings_dict"

# Imprimimos los primeros diez valores de los idMaps docs y terms, y también de postings_dict y termsId:
print("----------- terms: -----------")
print("10 primeros términos:         ", terms.id_to_str[:10])
print("10 primeros mappings term->id:", list(zip(list(terms.str_to_id.keys()),terms.str_to_id.values()))[:10])
print("\n--------- docs: ------------")
print("10 primeros documentos:       ", docs.id_to_str[:10])
print("10 primeros mappings doc->id: ", list(zip(list(docs.str_to_id.keys()),docs.str_to_id.values()))[:10])
print("\n------ postings_dict: ------")
print("10 primeros postings_dict:    ", list(zip(list(postings_dict.keys()),postings_dict.values()))[:10])
print("10 primeros termsID:          ", termsID[:10])

Leídos 98998 documentos y 347071 términos
----------- terms: -----------
10 primeros términos:          ['3d', 'radiology', 'lab', 'stanford', 'university', 'school', 'of', 'medicine', 'and', 'quantitative']
10 primeros mappings term->id: [('3d', 0), ('radiology', 1), ('lab', 2), ('stanford', 3), ('university', 4), ('school', 5), ('of', 6), ('medicine', 7), ('and', 8), ('quantitative', 9)]

--------- docs: ------------
10 primeros documentos:        ['0/3dradiology.stanford.edu_', '0/3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html', '0/3dradiology.stanford.edu_patient_care_case_studies.html', '0/5-sure.stanford.edu_', '0/50years.stanford.edu_', '0/a3cservices.stanford.edu_awards_nominate_', '0/a3cservices.stanford.edu_facilities_', '0/a3cservices.stanford.edu_lead_', '0/aa.stanford.edu_', '0/aa.stanford.edu_about_aviation.php']
10 primeros mappings doc->id:  [('0/3dradiology.stanford.edu_', 0), ('0/3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html', 1), ('

### Clase _Idf_

A continuación, la clase `Idf` que, a partir de los diccionarios anteriores, calcula la frecuencia (absoluta) de cada término, y el correspondiente valor IDF pesado logarítmicamente, como es habitual:

In [13]:
class Idf:
    """Construye un diccionario para poder devolver el IDF de un término (tanto si el término consultado está
       como si no está en el diccionario construido).
       Recuérdese de la práctica 1 que el diccionario "postings_dict" mapea cada termID a una tupla
       (posicion_de_comienzo_en_fichero_indice, numero_de_postings_en_la_lista, longitud_en_bytes_de_la_lista).
       Dado que queremos protegernos del caso posible de consulta de un término que no aparezca en la colección,
       aplicaremos el suavizado de Laplace típico (suma de +1 una unidad tanto en el numerador como en el
       denominador de la proporción, evitando así la posible división por cero).
    """
    def __init__(self):
        """Construcción del diccionario IDF"""
        try:
            with open("pa3-data/docs.dict", 'rb') as f:
                docs = pkl.load(f)
            self.total_doc_num = len(docs)
            print("Número total de documentos de la colección:", self.total_doc_num)

            with open("pa3-data/terms.dict", 'rb') as f:
                terms = pkl.load(f)
            self.total_term_num = len(terms)
            print("Número total de términos:", self.total_term_num)

            with open('pa3-data/BSBI.dict', 'rb') as f:
                postings_dict, termsID = pkl.load(f)

            self.idf = {}
            self.raw = {}
            for key in list(postings_dict.keys()):
                _, count, _ = postings_dict[key]
                # Frecuencias absolutas (raw):
                #### COMPLETA: Asigna a self.raw[terms[key]] = count
                self.raw[terms[key]] = count
                # Frecuencias pesadas logarítmicamente (usando suavizado Laplace):
                self.idf[terms[key]] = math.log10((1.0+self.total_doc_num) / (1.0+count))
        except FileNotFoundError:
            print("¡Ficheros de diccionario de documentos / términos / índice no encontrados!")

    def get_raw(self, term):
        """Devuelve el conteo crudo de ocurrencias de documentos conteniendo un término,
           dado, tanto si está como si no en el diccionario.
        Args:
            term(str) : Término del que se va a devolver su conteo crudo.
        Return(float):
            Idf del término.
        """
        
        try:
            return self.raw[term]
        except:
            # Para términos fuera del corpus, simplemente devolvemos 0:
            return 0

    def get_idf(self, term):
        """Devuelve el IDF de un término, tanto si está como si no en el diccionario.
        Args:
            term(str) : Término del que se va a devolver su IDF.
        Return(float):
            Idf del término.
        """
        ### BEGIN YOUR CODE (FIXME)
        try:
            return self.idf[term]
        except:
            # Para términos fuera del corpus, simplemente devolvemos math.log10((1.0+total_doc_num) / 1.0).
            return math.log10((1.0+self.total_doc_num) / 1.0)
        ### END YOUR CODE (FIXME)

Creamos ahora una instancia de esta clase `Idf`, y probamos a consultar en ella la IDF de unos cuantos términos, tanto existentes como inexistentes. Obsérvese que, a mayor frecuencia (términos más comunes en el corpus) de un término, menor peso IDF, y viceversa (términos menos comunes tienen un mayor peso IDF). Del mismo modo, un término inexistente tendrá un IDF muy cercano a 5.0 (ya que la cantidad total de documentos es aproximadamente 100K=$10^5$):

In [14]:
theIDF = Idf()
print()
print(f'IDF("the") =                  {theIDF.get_idf("the"):5.3f}  (raw count={theIDF.get_raw("the")})')
print(f'IDF("and") =                  {theIDF.get_idf("and"):5.3f}  (raw count={theIDF.get_raw("and")})')
print(f'IDF("stanford") =             {theIDF.get_idf("stanford"):5.3f}  (raw count={theIDF.get_raw("stanford")})')
print(f'IDF("university") =           {theIDF.get_idf("university"):5.3f}  (raw count={theIDF.get_raw("university")})')
print(f'IDF("quantitative") =         {theIDF.get_idf("quantitative"):5.3f}  (raw count={theIDF.get_raw("quantitative")})')
print(f'IDF("supercalifragilistic") = {theIDF.get_idf("supercalifragilistic"):5.3f}  (raw count={theIDF.get_raw("supercalifragilistic")})')

Número total de documentos de la colección: 98998
Número total de términos: 347071

IDF("the") =                  0.083  (raw count=81770)
IDF("and") =                  0.094  (raw count=79688)
IDF("stanford") =             0.143  (raw count=71202)
IDF("university") =           0.242  (raw count=56709)
IDF("quantitative") =         2.243  (raw count=565)
IDF("supercalifragilistic") = 4.996  (raw count=0)


Lo siguiente son unos cuantos tests adicionales, que aseguran la corrección de la implementación de la clase:

In [15]:
assert len(theIDF.idf) == 347071, 'Longitud incorrecta de diccionario idf.'
assert theIDF.get_idf("bilibalabulu") > 4.9, \
       "Término no localizado no manejado correctamente"
assert theIDF.get_idf("data") < theIDF.get_idf("radiology"), \
       "El idf de los términos más raros debe ser mayor que el de términos más comunes."
assert theIDF.get_idf("to") < theIDF.get_idf("design"), \
       "El idf de los términos más raros debe ser mayor que el de términos más comunes."
print("Tests de clase Idf() superados.")

Tests de clase Idf() superados.


# Vectores de conteo de términos

Cada par consulta-documento $(q,d)$ de los archivos de entrada proporciona información relativa a los diferentes términos de cada consulta en cinco campos diferentes de cada documento, a saber: <b>url</b>, <b>title</b>, <b>headers</b>, <b>body</b> y <b>anchors</b> (el campo <b>pagerank</b> adicional no se utilizará hasta más adelante).

Las funciones de _ranking_ construirán inicialmente los correspondientes cinco vectores de conteo de términos crudos ($rs$, de _raw score_) para cada uno de estos pares $(q,d)$, precisamente a partir de las coincidencias (_hits_) en estos cinco diferentes campos. Estos vectores $rs$ simplemente contarán cuantas veces ocurre cada término de búsqueda en un determinado campo. Para el campo <b>anchor</b>, se asumirá la simplificación de que hay un gran documento que contiene todos los enlaces, con el texto del enlace multiplicado por el campo <b>stanford_anchor_count</b>. Seguiremos un enfoque análogo también para el campo <b>header</b>.
    
Así, para el par $(q,d)$ de ejemplo donde la consulta era $q=[\text{stanford aoerc pool hours}]^T$ y el documento $d=$"http://events.stanford.edu/2014/February/18/" (mostrado en el ejemplo de una celda anterior de este mismo notebook), el vector ${rs}_{b}$ correspondiente al campo <b>body</b> será $[{10 \ 7 \ 1 \ 0}]^T$, dado que había 10 apariciones del término "stanford" en dicho campo, 7 para el término "aoerc", 1 para el término "pool" y ninguna para el término "hours". Análogamente, el vector ${rs}_{a}$ para el campo <b>anchor</b> será simplemente $[\text{0 0 0 0}]^T$, dado que no hay ningún enlace (<b>anchor</b>) para este documento. Finalmente el vector ${rs}_{t}$ para el campo <b>title</b> será $[\text{1 0 0 0}]^T$, $[\text{1 0 0 0}]^T$ también para el vector ${rs}_{u}$ del campo <b>url</b> (estos dos fácilmente deducibles desde los correspondientes campos `title` y `url`, que son únicos en cada documento), y ${rs}_{h}$ $[\text{5 0 0 1}]^T$ para el campo <b>header</b> (este último acumulado para las distintas cabeceras --esto es, "subtítulos"-- presentes en el documento). Todo esto se puede corroborar fácilmente observando el contenido de dicho documento en el archivo de señal original:
    
```   
query: stanford aoerc pool hours
 url: http://events.stanford.edu/2014/February/18/
  title: events at stanford tuesday february 18 2014
  header: stanford university event calendar
  header: teaching sex at stanford
  header: rodin the complete stanford collection
  header: stanford rec trx suspension training
  header: memorial church open visiting hours
  header: alternative transportation counseling tm 3 hour stanford univ shc employees retirees family members
  body_hits: stanford 239 271 318 457 615 642 663 960 966 971
  body_hits: aoerc 349 401 432 530 549 578 596
  body_hits: pool 521
  body_length: 981
  pagerank: 1    
```
    
Un ejemplo adicional, usando la misma consulta, pero un documento de respuesta diferente, para ilustrar los vectores correspondientes a los campos <b>anchor</b>, no presentes en el documento anterior:
```
  url: https://cardinalrec.stanford.edu/facilities/aoerc/
    ...
    anchor_text: gyms aoerc
      stanford_anchor_count: 3
    anchor_text: aoerc
      stanford_anchor_count: 13
    anchor_text: http cardinalrec stanford edu facilities aoerc
      stanford_anchor_count: 4
    anchor_text: arrillaga outdoor education and recreation center aoerc link is external
      stanford_anchor_count: 1
    anchor_text: the arrillaga outdoor education and research center aoerc
      stanford_anchor_count: 2
    anchor_text: aoerc will shutdown for maintenance
      stanford_anchor_count: 2
```

Aquí, el vector para  <b>anchor</b> será $[\text{4 25 0 0}]^T$, dado que sólo hay un total de 4 enlaces (campo <b>stanford_anchor_count</b>) para el término "stanford", pero 25 (=3+13+4+1+2+2) para el término “aoerc”.

(Nótese que, en lo referente al campo  <b>url</b> es necesario _"tokenizar"_ previamente sólo los caracteres alfanuméricos. Nótese también que, al calcular los conteos de términos crudos, todo se hace convirtiendo previamente los _tokens_ a minúsculas).

# Clase base para todos los _scorers_

# Clase _AbstractScorer_

Construimos ahora una clase base abstracta, de la que habrá que ir reimplementando métodos en las clases hijas, conforme vayamos implementando los diferentes métodos de _scoring_. En todo caso, esta clase AbstractScorer recogerá una funcionalidad básica común a todos. Lo fundamental serán los distintos métodos `parse_`_field_, que transforman la información textual disponible en el archivo de señal para cada campo en la correspondiente información numérica. Aprovechamos también aquí para añadir los posibles esquemas de _weighting_ SMART vistos en teoría (si bien en esta práctica no programaremos todas las posibilidades disponibles, sino sólo alguna de las más comúnmente utilizadas):

In [ ]:
class AbstractScorer:
    """ Una clase básica abstracta para un scorer.
        Implementa una funcionalidad básica de construcción de vectores de consulta y de documento.
        Tendrá que ser extendida adecuadamente por cada scorer específico.
    """
    def __init__(self, idf, query_weight_scheme=None, doc_weight_scheme=None):
        self.idf = idf
        self.TFTYPES = ["url", "title", "body_hits", "header", "anchor"]
        # Esquemas por defecto:
        self.default_query_weight_scheme = {"tf": 'n', "df": 'n', "norm": None} # Esquema natural, none, none
        self.default_doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": None}   # Esquema natural, none, none
        self.query_weight_scheme = query_weight_scheme if query_weight_scheme is not None \
                                   else self.default_query_weight_scheme
        self.doc_weight_scheme = doc_weight_scheme if doc_weight_scheme is not None \
                                 else self.default_doc_weight_scheme

    def parse_url(self, url):
        """Parsea la URL del documento, devolviendo un Counter de los tokens encontrados en la URL.
        Args:
            url: el url del que se va a hacer el parsing.
        Returns:
            Lista de tokens del URL (una vez limpios), y Counter resultado.
        """
        if url:
            url_token_in_term = url.replace("http:",".").replace('/','.').replace('?','.') \
                                   .replace('=','.').replace("%20",".").replace("...",".").replace("..",".")\
                                   .replace('-','.').lower();
            url_token = url_token_in_term.strip(".").split('.')
            return url_token, Counter(url_token)
        else:
            return [], Counter([])

    def parse_title(self, title):
        """Parsea el campo title del documento, devolviendo un Counter de los tokens encontrados en el mismo.
        Args:
            title: el title del que se va a hacer el parsing.
        Returns:
            El Counter resultado.
        """
        if title:
            return Counter(title.split(" "))
        else:
            return Counter([])

    def parse_headers(self, headers):
        """Parsea los campos headers del documento, devolviendo un Counter de los tokens encontrados en los mismos.
        Args:
            headers: la lista de headers sobre los que se va a hacer el parsing.
        Returns:
            El Counter resultado.
        """
        headers_token = []
        if headers is not None:
            for header in headers:
                header_token = header.split(" ")
                headers_token.extend(header_token)
        return Counter(headers_token)

    def parse_anchors(self, anchors):
        """Parsea los campos anchors del documento, devolviendo un Counter de los tokens encontrados en los mismos.
        Args:
            anchors: la lista de anchors sobre los que se va a hacer el parsing.
        Returns:
            El Counter resultado.
        """
        anchor_count_map = Counter({})
        if anchors is not None:
            for anchor in anchors:
                count = anchors[anchor]
                anchor_tokens = anchor.split(" ")
                for anchor_token in anchor_tokens:
                    if(anchor_token in anchor_count_map.keys()):
                        anchor_count_map[anchor_token] += count
                    else:
                        anchor_count_map[anchor_token] = count
        return anchor_count_map

    def parse_body_hits(self, body_hits):
        """Parsea los campos body_hits del documento, devolviendo un Counter de los tokens encontrados en los mismos.
        Args:
            body_hits: la lista de elementos del body.
        Returns:
            El Counter resultado.
        """
        body_hits_count_map = Counter({})
        if body_hits is not None:
            for body_hit in body_hits:
                body_hits_count_map[body_hit] = len(body_hits[body_hit])
        return body_hits_count_map

    def get_query_vector(self, q, query_weight_scheme = None):
        """ Obtiene un vector numérico para la consulta q.
        Args:
            q (Query): Query("Una consulta determinada")
        Returns:
            query_vec (dict): El vector resultado.
        """
        # En subclases de esta AbstractScorer, podrían tenerse en cuenta todas las
        # posibilidades SMART, usando diferentes esquemas de frecuencia del término (tf),
        # frecuencia de documento (idf) y normalización. En todo caso, nótese que en
        # general no se suele necesitar normalización para la consulta en ningún caso, ya que
        # dicha normalización no variaría con respecto a todos los documentos resultados de una
        # misma consulta, lo que resultaría en un simple factor de escalado común que no
        # afectaría al posterior ranking de los mismos.
        #
        # if query_weight_scheme is None:
        #     query_weight_scheme = self.query_weight_scheme

        query_vec = {}
        ### BEGIN YOUR CODE (FIXME)
        # En nuestro caso base, usaremos simplemente el contador básico de términos, sin
        # normalización ni uso de idf:
        query_vec = Counter(q.query_words)
        ### END YOUR CODE (FIXME)
        return query_vec

    def get_doc_vector(self, q, d, doc_weight_scheme=None):
        """ Obtiene un vector numérico para el documento d.
        Args:
        q (Query) : Query("Una consulta")
        d (Document) : Query("Una consulta")["Un URL"]
        Returns:
        doc_vec (dict) : Un diccionario de conteo de la frecuencia de términos, con un subdiccionario para
                         cada tipo de campo (tipo_de_campo -> (término -> conteo))
                    Ejemplo: "{'url':   {'stanford': 1, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               'title': {'stanford': 1, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               ...
                               }"
        """
        # De nuevo, podrían considerarse todas las posibilidades SMART en las subclases
        # de esta AbstractScorer, si bien en esta clase base nos contentaremos con un simple
        # conteo crudo de los términos en los distintos campos:
        #
        # if doc_weight_scheme is None:
        #    doc_weight_scheme = self.doc_weight_scheme

        doc_vec = {}
        ### BEGIN YOUR CODE (FIXME)
        # Sólo para depurar:
        # print(f"URL:        {d.url}          ->   {self.parse_url(d.url)}")
        # print(f"TITLE:      {d.title}        ->   {self.parse_title(d.title)}")
        # print(f"HEADERS:    {d.headers}      ->   {self.parse_headers(d.headers)}")
        # print(f"ANCHORS:    {d.anchors}      ->   {self.parse_anchors(d.anchors)}")
        # print(f"BODY_HITS:  {d.body_hits}    ->   {self.parse_body_hits(d.body_hits)}")
        #
        # Simple conteo crudo de los términos por campos:
        _, doc_vec["url"] = self.parse_url(d.url)
        doc_vec["title"] = self.parse_title(d.title)
        doc_vec["headers"] = self.parse_headers(d.headers)
        doc_vec["anchors"] = self.parse_anchors(d.anchors)
        doc_vec["body_hits"] = self.parse_body_hits(d.body_hits)
        ### END YOUR CODE (FIXME)
        return doc_vec

    # Métodos no implementados en la clase base; en su caso, serán reimplementados en cada scorer concreto:

    def normalize_doc_vec(self, q, d, doc_vec):
        """ Normalizar el vector de documento.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            doc_vec (dict) : El vector de documento
        """
        raise NotImplementedError

    def get_sim_score(self, q, d):
        """ Devuelve la puntuación para una consulta q y documento d dados.
        Args:
            q (Query): la consulta.
            d (Document) : el documento.
        Returns:
            La puntuación para el par (q,d).
        """
        raise NotImplementedError

    def get_net_score(self, q, d):
        """ Calcular el scoring neto entre la consulta y el documento.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
        Return:
            score (float) : La puntuación resultado.
        """
        raise NotImplementedError

Probamos la clase abstracta con una consulta y un documento:

In [ ]:
q = Query("stanford aoerc pool hours")
d = query_dict[q]['http://events.stanford.edu/2014/February/18/']
print("Query q: ", q)
print()
print("Document d: ", d)

a_scorer = AbstractScorer(theIDF)
query_vec = a_scorer.get_query_vector(q)
print(f"Vector consulta:")
print(f"  {query_vec}")
print()
doc_vec = a_scorer.get_doc_vector(q, d)
print(f"Vector documento:")
for k, v in doc_vec.items():
    print(f"  {k:10s} -> {v}")

Query q:  stanford aoerc pool hours

Document d:  url: http://events.stanford.edu/2014/February/18/
 title: events at stanford tuesday february 18 2014
 headers: ['stanford university event calendar', 'teaching sex at stanford', 'rodin the complete stanford collection', 'stanford rec trx suspension training', 'memorial church open visiting hours', 'alternative transportation counseling tm 3 hour stanford univ shc employees retirees family members']
 body_hits: {'stanford': [239, 271, 318, 457, 615, 642, 663, 960, 966, 971], 'aoerc': [349, 401, 432, 530, 549, 578, 596], 'pool': [521]}
 body_length: 981
 pagerank: 1

Vector consulta:
  Counter({'stanford': 1, 'aoerc': 1, 'pool': 1, 'hours': 1})

Vector documento:
  url        -> Counter({'events': 1, 'stanford': 1, 'edu': 1, '2014': 1, 'february': 1, '18': 1})
  title      -> Counter({'events': 1, 'at': 1, 'stanford': 1, 'tuesday': 1, 'february': 1, '18': 1, '2014': 1})
  headers    -> Counter({'stanford': 5, 'university': 1, 'event': 1,

## Clase _BaselineScorer_

Definimos aquí un simple _"scorer baseline"_, que nos servirá para probar la funcionalidad de la clase abstracta base. La clase `BaselineScorer` heredará directamente de la clase `AbstractScorer`, reimplementando solamente el método `get_sim_score`, que simplemente acumulará, para aquellos términos en la consulta, los contadores absolutos de dichos términos en el vector de documento correspondiente, utilizando en cada caso el campo `url`, `title`, `headers`, `anchors`, o `body_hits` que se le indique en cada momento mediante el método `set_field_type(...)`:

In [ ]:
class BaselineScorer(AbstractScorer):
    def __init__(self, idf):
        super().__init__(idf)
        self.field_type = "url"  # Campo por defecto de inicialización de la clase.

    def set_field_type(self, field_type):
        self.field_type = field_type

    def get_sim_score(self, q, d):
        q_vec = self.get_query_vector(q)
        d_vec = self.get_doc_vector(q, d)
        score = 0
        if self.field_type in d_vec.keys():
            for term in d_vec[self.field_type].keys():
                if term in q_vec.keys():
                    score += d_vec[self.field_type][term]
        return score

    # En este caso el scoring neto coincide con el devuelto por get_sim_score
    # (no se combinan los distintos campos, simplemente se tiene en cuenta el
    #  campo fijado previamente con set_field_type):
    def get_net_score(self, q, d):
        return self.get_sim_score(q, d)

Probamos el _baseline scorer_ con una consulta y un par de documentos de ejemplo, para todos los campos posibles:

In [ ]:
baseline_scorer = BaselineScorer(theIDF)

q = Query("stanford aoerc pool hours")
print("---------\n")
print("Consulta: ", q)
print(f"Vector de consulta:\n{baseline_scorer.get_query_vector(q)}")
print("\n---------\n")

d1 = query_dict[q]['http://events.stanford.edu/2014/February/18/']          # Ejemplo que tiene "body_hits".
d2 = query_dict[q]['https://cardinalrec.stanford.edu/facilities/aoerc/']    # Ejemplo que tiene "anchors".

for i,d in enumerate([d1,d2]):
    doc_vectors = baseline_scorer.get_doc_vector(q,d)
    print("Documento:\n", d)
    print(f"Vectores de documento:")
    similarities = {}
    for k in doc_vectors.keys(): # Para cada campo:
        baseline_scorer.set_field_type(k)
        similarity = baseline_scorer.get_sim_score(q,d)
        print(f"  {k} vector (computed similarity={similarity}):\n  {doc_vectors[k]}\n")
        similarities[k] = similarity
    # print(similarities)
    if i==0:
        assert similarities == {'url': 1, 'title': 1, 'headers': 6, 'anchors': 0, 'body_hits': 18}, \
          "Scorer de similaridad baseline utilizando pesos por defecto no obtiene resultado esperado para d1"
    elif i==1:
        assert similarities == {'url': 2, 'title': 0, 'headers': 0, 'anchors': 29, 'body_hits': 0}, \
          "Scorer de similaridad baseline utilizando pesos por defecto no obtiene resultado esperado para d2"
    print("---------\n")

print("Tests de clase BaselineScorer() superados.")

---------

Consulta:  stanford aoerc pool hours
Vector de consulta:
Counter({'stanford': 1, 'aoerc': 1, 'pool': 1, 'hours': 1})

---------

Documento:
 url: http://events.stanford.edu/2014/February/18/
 title: events at stanford tuesday february 18 2014
 headers: ['stanford university event calendar', 'teaching sex at stanford', 'rodin the complete stanford collection', 'stanford rec trx suspension training', 'memorial church open visiting hours', 'alternative transportation counseling tm 3 hour stanford univ shc employees retirees family members']
 body_hits: {'stanford': [239, 271, 318, 457, 615, 642, 663, 960, 966, 971], 'aoerc': [349, 401, 432, 530, 549, 578, 596], 'pool': [521]}
 body_length: 981
 pagerank: 1

Vectores de documento:
  url vector (computed similarity=1):
  Counter({'events': 1, 'stanford': 1, 'edu': 1, '2014': 1, 'february': 1, '18': 1})

  title vector (computed similarity=1):
  Counter({'events': 1, 'at': 1, 'stanford': 1, 'tuesday': 1, 'february': 1, '18': 1, '2

# Función de _ranking_ 1: Similaridad del coseno

Mas allá del _scoring_ _baseline_ (un tanto _naive_) anterior, la primera función de _ranking_ más elaborada que aplicaremos consistirá en una variante clásica de la similitud coseno (con la norma L1). Se trata esencialmente de construir el vector del documento y el vector de la consulta para luego tomar el producto escalar como resultado.
En realidad, es exactamente lo mismo que hemos hecho ya para el simple conteo de términos anterior (suponiendo que el vector consulta era siempre un vector binario con 1 en los términos contenidos en la consulta, y 0 en el resto). Ahora, sin embargo, en lugar de tomar simplemente los conteos de términos (tanto en la consulta como en el documento), podrían considerarse varias alternativas para calcular el peso (=componente) de cada término, decidiendo:

1. Cómo se calcula exactamente la frecuencia de cada término.
2. Cómo se realiza la ponderación por frecuencia de documento de cada término.
3. La estrategia de normalización seguida.

De nuevo, la figura 6.15 de la página 128 del libro de Manning ([enlace](http://nlp.stanford.edu/IR-book/pdf/06vect.pdf)) nos recuerda todas las posibles alternativas para ello, según la notación SMART.

En lo que sigue discutiremos las opciones para ambos vectores por separado.

## Vectores de consulta

* **Frecuencia del término** (_tf_):
Las frecuencias crudas pueden computarse de forma sencilla a partir de los términos de la consulta. Deberían corresponderse, en la mayoría de los casos, con un simple 1 para cada término que apareciese en la consulta, equivalente a la opción _"boolean"_ (pero no necesariamente, ya que algún término podría aparecer repetido). Dicha frecuencia cruda podría, si se desease, ser escalada sublinealmente (usando el logaritmo). En todo caso, en este notebook mantendremos el enfoque simple del conteo natural de términos, muy similar al vector booleano, dado que la inmensa mayoría de las consultas del _dataset_ no contienen términos repetidos.

* **Frecuencia del documento** (_df_):
Cada uno de los términos en el vector de la consulta deberá entonces ser pesado (=multiplicado) usando el valor de IDF correspondiente para cada término. Usaremos, como ya se comentó antes en este mismo notebook, el IDF computado a partir del corpus de la práctica 1. Recuérdese que, para el caso de palabras que no apareciesen en dicho corpus, se usará la técnica del suavizado Laplaciano (es decir, simplemente sumar 1 en el numerador y en el denominador; esto equivale esencialmente a asumir la existencia de un hipotético documento _"dummy"_ que contiene todos los posibles términos)

* **Normalización** (_norm_):
En ningún caso será necesario normalizar el vector de consulta, ya que cualquier posible normalización se aplicaría por igual al cruzarlo con todos los correspondientes documentos resultado, obteniéndose un simple escalado uniforme de los valores de _scoring_, lo que obviamente no influiría en absoluto en el correspondiente _ranking_ de resultados.

## Vectores de documento

* **Frecuencia del término** (_tf_):
Al igual que con el vector de consulta, podremos utilizar directamente las frecuencias crudas, o, alternativamente, aplicar algún tipo de escalado sublineal. En particular, el escalado sublineal típico es $tf_i = 1 + log(rs_i)$ si $rs_i > 0$, o simplemente $0$ en caso contrario. Así, por ejemplo, para el anterior vector _tf_ del campo **body** del documento _d_, el vector resultante sería $[\text{1+log(10)  1+log(7)  1+log(1)  0}]^T$ (de nuevo, puede encontrarse más información sobre el escalado sublineal del término _tf_ en la página 126, sección 6.4.1 del [libro de Manning](http://nlp.stanford.edu/IR-book/pdf/06vect.pdf)).

* **Frecuencia del documento** (_df_):
No utilizaremos ningún tipo de frecuencia del documento en el vector de documento. En lugar de ello, se incorporará este peso únicamente en el vector de consulta, como se describía en el apartado anterior.

* **Normalización** (_norm_):
En este caso, no podemos usar la normalización del coseno, dado que nuestros archivos de entrenamiento no proporcionan acceso al contenido del documento en sí, sino solo un resumen de campos. Por lo tanto, no sabemos ni qué otros términos aparecen, ni el recuento de los mismos, en el campo **body**. En lugar de ello, por tanto, utilizaremos la normalización de longitud. Además, dado que puede haber enormes discrepancias entre las longitudes de los diferentes campos, dividimos todos los campos por el mismo factor de normalización, la propia longitud del campo **body**. Dado, además, el hecho de que algunos documentos aparecen con una longitud de 0, una buena estrategia es, de nuevo, agregar un valor (p.e. 500), a la longitud del cuerpo de cada documento. El valor concreto a utilizar, además, puede ser también utilizado para experimentar con ésta u otras estrategias de suavizado, y observar su posible influencia en los resultados de _ranking_ obtenidos.

## Pesado relativo de los diferentes campos

Dado un documento $d$ y una consulta $q$, si $qv_q$ es el vector resultante de la consulta y $tf_{d,u}$, $tf_{d,t}$, $tf_{d,b}$, $tf_{d,h}$ y $tf_{d,a}$ son los vectores resultantes para cada uno de los campos **url**, **title**, **body**, **header** and **anchor**, respectivamente, definimos el _scoring_ global neto como (nótese que el símbolo $\cdot$ se usa en la siguiente expresión tanto para el producto escalar entre vectores como para el producto de un escalar por un vector):

$$qv_q \cdot (c_u \cdot tf_{d,u} + c_t \cdot tf_{d,t} + c_b \cdot tf_{d,b} + c_h \cdot tf_{d,h} + c_a \cdot tf_{d,a})$$

Donde $c_u$, $c_t$, $c_b$, $c_h$ y $c_a$ son los pesos dados a los campos **url**, **title**, **body**, **header** y **anchor**, respectivamente.

Por supuesto, para usar la expresión anterior necesitamos determinar de una forma sensata los pesos para cada uno de estos cinco campos. En este sentido, trataremos de escogerlos de forma que la función NDCG de evaluación (que describiremos más adelante) obtenga un valor lo más optimizado posible cuando sea aplicada al conjunto de test completo. Usaremos el conjunto de _training_ para intentar derivar dicho valor óptimo de los cinco parámetros mencionados, para luego evaluar su rendimiento en el conjunto de _test_. En una primera instancia, lo intentaremos hacer manualmente, intentando razonar simplemente sobre la importancia relativa de los diferentes pesos. Al final del notebook sustituiremos esta suerte de "razonamiento manual" por una śencilla técnica de _machine learning_.

Nótese que el valor absoluto de dichos pesos no importará, sólo la relación (ratio) entre ellos (valores relativos), dado que si multiplicamos todos los pesos por la misma constante, el _scoring_ final quedará simplemente multiplicado por dicha constante para todos los documentos por igual, lo que obviamente no afectará en absoluto a la ordenación.

### Esquema de _weighting_ inicial

Proporcionamos aquí un esquema de pesado por defecto de partida, que puede después variarse para intentar mejorar el rendimiento (medido con NDGC). Nótese que:
* En estos primeros pesos por defecto se asigna una importancia del peso de la URL 100 veces superior a la del resto de pesos, a los que, por otro lado, se da un peso equivalente.
* Se añade al conjunto de parámetros ajustables un parámetro de suavizado de la longitud del documento (`smoothing_body_length`), término que podrá ser modificado para influir en la función de _scoring_ final. Dicho término será simplemente sumado a la longitud original de cada documento (con lo que, en todo caso, y como se comentó anteriormente, se evitará siempre la posible división por cero para documentos en los cuales la longitud indicada en el documento del dataset de entrada sea cero). Se deduce, pues, que dar un valor cada vez mayor para este parámetro implicará una influencia progresivamente menor del campo `body_length` original de cada documento particular, ya que el correspondiente factor de influencia de la longitud tenderá con ello a homogeneizarse más para todos los documentos, al disminuir progresivamente el peso relativo del valor inicial de `body_length` en la suma total del denominador.

In [ ]:
params_cosine = {
    "url_weight" : 10,
    "title_weight": 0.1,
    "body_hits_weight" : 0.1,
    "header_weight" : 0.1,
    "anchor_weight" : 0.1,
    "smoothing_body_length" : 500,
}

## Clase _CosineSimilarityScorer_

He aquí la definición de una clase para realizar un _scoring_ basado en la similaridad del coseno (basada en la clase `AbstractScorer` definida anteriormente, y reimplementando los métodos adecuados):

In [ ]:
class CosineSimilarityScorer(AbstractScorer):

    def __init__(self, idf, query_dict, params, query_weight_scheme=None, doc_weight_scheme=None):
        # Inicializamos clase base "AbstractScorer", ...
        super().__init__(idf, query_weight_scheme=query_weight_scheme, doc_weight_scheme=doc_weight_scheme)
        self.query_dict = query_dict
        # ... y añadimos los parámetros necesarios (5 pesos de los 5 campos + factor suavizado longitud):
        try:
            self.smoothing_body_length = params["smoothing_body_length"]
        except:
            self.smoothing_body_length = 0
        self.weights = {"url": params["url_weight"], "title": params["title_weight"],
                        "headers": params["header_weight"], "anchors": params["anchor_weight"],
                        "body_hits": params["body_hits_weight"]}

    def get_query_vector(self, q):
        """ Usando los vectores de conteo crudos de la clase base, aplica diferentes variantes
            SMART para obtener el correspondiente vector numéricos de consulta modificado.
        Args:
            q (Query): Query("Una consulta determinada")
        Returns:
            query_vec (dict): El vector resultado.
        """
        # Método de conteo de la clase base:
        query_vec = super().get_query_vector(q)

        # Frecuencia de documento (implementadas las variantes n, b, y t SMART):
        if self.query_weight_scheme["df"] == "n":     # No se modifica query_vec:
            pass
        elif self.query_weight_scheme["df"] == "b":   # Vector query_vec booleano:
            for key_term in query_vec.keys():
                query_vec[key_term] = 1
        elif self.query_weight_scheme["df"] == "t":   # Modificación IDF de query_vec:
            for key_term in query_vec.keys():
                query_vec[key_term] = self.idf.get_idf(key_term)

        return query_vec

    def get_doc_vector(self, q, d):
        """ Usando los vectores de conteo crudos de la clase base, aplica diferentes variantes
            SMART para obtener los correspondientes vectores numéricos de documento modificados.
        Args:
        q (Query) : Query("Una consulta")
        d (Document) : Query("Una consulta")["Un URL"]
        Returns:
        doc_vec (dict) : Vectores numéricos modificados, de nuevo con esquema (tipo_de_campo -> (término -> conteo))
                    Ejemplo: "{'url':   {'stanford': 0.13, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               'title': {'stanford': 0.11, 'aoerc': 0, 'pool': 0, 'hours': 0},
                               ...
                               }"
        """
        # Método de conteo de la clase base:
        doc_vec = super().get_doc_vector(q, d)

        # Frecuencia de término (implementadas las variantes n y l SMART):
        if self.doc_weight_scheme["tf"] == "n":   # No se modifica doc_vec:
            pass
        elif self.doc_weight_scheme["tf"] == "l": # Modificación logarítmica (sublineal) de doc_vec
            for key_field in doc_vec.keys():
                for key_term in doc_vec[key_field].keys():
                    doc_vec[key_field][key_term] = 1 + math.log(doc_vec[key_field][key_term])

        # Normalización:
        if self.doc_weight_scheme['norm'] == "default":
            doc_vec = self.normalize_doc_vec(q, d, doc_vec)

        return doc_vec

    def get_sim_score(self, q, d, field_type):
        """ Cálculo del scoring para un campo individual:
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            field_type (str) : El campo del que se usará el vector de documento.
        Return:
            score (float) : El scoring individual (para el campo field_type) del par (q,d):
        """
        ### BEGIN YOUR CODE (FIXME)
        q_vec = self.get_query_vector(q)
        d_vec = self.get_doc_vector(q, d)
        score = 0
        if field_type in d_vec.keys():
            for term in d_vec[field_type].keys():
                if term in q_vec.keys():
                    score += d_vec[field_type][term] * q_vec[term]
        ### END YOUR CODE (FIXME)
        return score

    def get_net_score(self, q, d):
        """ Cálculo del scoring global (neto), usando los cinco pesos:
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
        Return:
            score (float) : El scoring global (neto, sumando todos los campos) del par (q,d):
        """
        ### BEGIN YOUR CODE (FIXME)
        score = 0
        for field_type in ["url", "title", "headers", "anchors", "body_hits"]:
            score +=  self.weights[field_type] * self.get_sim_score(q, d, field_type)
        ### END YOUR CODE (FIXME)
        return score

    ## Normalización
    def normalize_doc_vec(self, q, d, doc_vec):
        """ Normalización del vector de documento:
        Damos una normalización uniforme basada en la longitud del documento, tal
        y como se discutió en el item "Normalización" del anterior apartado
        "Vectores de documento".
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            doc_vec (dict) : El vector de documento.
        Return:
            doc_vec (dict) : El vector de documento tras la normalización.
        """
        ### BEGIN YOUR CODE (FIXME)
        for key_field in doc_vec.keys():
            for key_term in doc_vec[key_field].keys():
                doc_vec[key_field][key_term] /= (d.body_length + self.smoothing_body_length)
        ### END YOUR CODE (FIXME)
        # print(d.body_length, self.smoothing_body_length)

        return doc_vec

He aquí una primera prueba sencilla de _scoring_ de un par ($q$,$d$) usando esta similaridad del coseno, en particular usando el esquema SMART _ddd.qqq_ = _nnn.bnn_:

In [ ]:
q = Query("stanford aoerc pool hours")
d = query_dict[q]['http://events.stanford.edu/2014/February/18/']
doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": None}
query_weight_scheme = {"tf": 'b', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector consulta: ', cs.get_query_vector(q), '\n')
print('Vector de documento original:\n', cs.get_doc_vector(q, d), '\n')
print("---")
print("Scoring campo url:", cs.get_sim_score(q,d,"url"))
print("Scoring campo title:", cs.get_sim_score(q,d,"title"))
print("Scoring campo headers:", cs.get_sim_score(q,d,"headers"))
print("Scoring campo anchors:", cs.get_sim_score(q,d,"anchors"))
print("Scoring campo body_hits:", cs.get_sim_score(q,d,"body_hits"))
print("\nScoring neto:", cs.get_net_score(q,d))

Vector consulta:  Counter({'stanford': 1, 'aoerc': 1, 'pool': 1, 'hours': 1}) 

Vector de documento original:
 {'url': Counter({'events': 1, 'stanford': 1, 'edu': 1, '2014': 1, 'february': 1, '18': 1}), 'title': Counter({'events': 1, 'at': 1, 'stanford': 1, 'tuesday': 1, 'february': 1, '18': 1, '2014': 1}), 'headers': Counter({'stanford': 5, 'university': 1, 'event': 1, 'calendar': 1, 'teaching': 1, 'sex': 1, 'at': 1, 'rodin': 1, 'the': 1, 'complete': 1, 'collection': 1, 'rec': 1, 'trx': 1, 'suspension': 1, 'training': 1, 'memorial': 1, 'church': 1, 'open': 1, 'visiting': 1, 'hours': 1, 'alternative': 1, 'transportation': 1, 'counseling': 1, 'tm': 1, '3': 1, 'hour': 1, 'univ': 1, 'shc': 1, 'employees': 1, 'retirees': 1, 'family': 1, 'members': 1}), 'anchors': Counter(), 'body_hits': Counter({'stanford': 10, 'aoerc': 7, 'pool': 1})} 

---
Scoring campo url: 1
Scoring campo title: 1
Scoring campo headers: 6
Scoring campo anchors: 0
Scoring campo body_hits: 18

Scoring neto: 12.5


Y he aquí, para el mismo par ($q$,$d$), algunos posibles vectores alternativos, obtenidos usando diferentes variantes SMART, tanto para la consulta $q$ como para el documento $d$:

In [ ]:
q = Query("stanford aoerc pool hours")
d = query_dict[q]['http://events.stanford.edu/2014/February/18/']

query_weight_scheme, doc_weight_scheme = None, None

query_weight_scheme = {"tf": 'b', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector consulta original: ', cs.get_query_vector(q), '\n')

query_weight_scheme = {"tf": 'b', "df": 't', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector consulta IDF: ', cs.get_query_vector(q), '\n')

doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento original:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": "default"}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento normalizado:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

doc_weight_scheme = {"tf": 'l', "df": 'n', "norm": None}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento con escalado logarítmico de tf:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

doc_weight_scheme = {"tf": 'l', "df": 'n', "norm": "default"}
cs = CosineSimilarityScorer(theIDF, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
print('Vector de documento con escalado logarítmico y normalizado:\n', cs.get_doc_vector(q, d), '\n')
print("-----")

Vector consulta original:  Counter({'stanford': 1, 'aoerc': 1, 'pool': 1, 'hours': 1}) 

Vector consulta IDF:  Counter({'aoerc': 4.995630807762446, 'pool': 2.446627545736658, 'hours': 1.2882309766291968, 'stanford': 0.14313251558629017}) 

Vector de documento original:
 {'url': Counter({'events': 1, 'stanford': 1, 'edu': 1, '2014': 1, 'february': 1, '18': 1}), 'title': Counter({'events': 1, 'at': 1, 'stanford': 1, 'tuesday': 1, 'february': 1, '18': 1, '2014': 1}), 'headers': Counter({'stanford': 5, 'university': 1, 'event': 1, 'calendar': 1, 'teaching': 1, 'sex': 1, 'at': 1, 'rodin': 1, 'the': 1, 'complete': 1, 'collection': 1, 'rec': 1, 'trx': 1, 'suspension': 1, 'training': 1, 'memorial': 1, 'church': 1, 'open': 1, 'visiting': 1, 'hours': 1, 'alternative': 1, 'transportation': 1, 'counseling': 1, 'tm': 1, '3': 1, 'hour': 1, 'univ': 1, 'shc': 1, 'employees': 1, 'retirees': 1, 'family': 1, 'members': 1}), 'anchors': Counter(), 'body_hits': Counter({'stanford': 10, 'aoerc': 7, 'pool': 1

# Función de _ranking_ 2: BM25F

Usaremos aquí la frecuencia de términos normalizada por campos ($ftf$, de _Field dependent normalized Term Frequency_). Así, para un término $t$ dado, y un campo $f \in \{url, header, body, title, anchor\}$ en el documento $d$, usaremos:

\begin{equation}
ftf_{d,f,t} = \frac{tf_{d,f,t}}{1 + B_f((\text{len}_{d,f} / \text{avlen}_f) - 1)}
\tag{1}
\end{equation}

Donde $tf_{d,f,t}$ es la frecuencia cruda de $t$ en el campo $f$ del documento $d$, $len_{d,f}$ es la longitud del campo $f$ en $d$, y $avlen_f$ es la longitud media del mismo campo en toda la colección.

Por supuesto, las correspondientes variables $avlen_{body}$, $avlen_{url}$, $avlen_{title}$, $avlen_{header}$ y $avlen_{anchor}$ se deberán computar usando de nuevo el conjunto de _training_. Los valores de $B_f$ serán parámetros adicionales dependientes de cada uno de los campos $f$, y al igual que los $c_f$ de la sección anterior, deberán ser ajustados. Si $avlen_f$ fuese cero, entonces definiremos $ftf_{d,f,t} = 0$ (si bien esto no debería ser necesario en este _dataset_ en concreto).

El peso global para el término $t$ en el documento $d$, usando ya todos los campos, sería:

\begin{equation}
w_{d,t} = \sum_{f} W_f \cdot ftf_{d,f,t}
\tag{2}
\end{equation}

Siendo, de nuevo, los distintos $W_f$ parámetros que determinan los pesos de importancia relativos dados a cada uno de los campos.

Puesto que, además, tenemos también una característica adicional no textual (el <b>pagerank</b>), la incorporaremos también en nuestra función de _ranking_ usando el método descrito en las transparencias de teoría.

En concreto, pues, y resumiendo, el _scoring_ global para el $d$ respecto a la consulta $q$ quedará definido como:

\begin{equation}
\sum_{t \in q} \frac{w_{d,t}}{K_1 + w_{d,t}}idf_t + \lambda V_{j}(f)
\tag{3}
\end{equation}

De nuevo aquí $K_1$ es un parámetro libre, y la función $V_{j}$ podría ser cualquier de las funciones logarítmicas, de saturación o sigmoide mencionadas en las transparencias de teoría, y que en este caso fijaremos simplemente como $V_{pagerank}(pr) = log(\lambda'+pr)$.

$\lambda$ y $\lambda'$ son los dos últimos parámetros libres adicionales para este modelo.

## Clase _BM25FScorer_

Definimos aquí la clase `BM25FScorer`, basada en la `CosineSimilarityScorer` anterior, pero incorporando en este caso todas las modificaciones necesarias para implementar la nueva funcionalidad descrita en los párrafos anteriores:

In [ ]:
class BM25FScorer(CosineSimilarityScorer):
    def __init__(self, idf, query_dict, params, query_weight_scheme=None, doc_weight_scheme=None):
        super().__init__(idf, query_dict, params, query_weight_scheme=query_weight_scheme, doc_weight_scheme=doc_weight_scheme)

        # Añadimos aquí los pesos ya específicos para BM25, y los nuevos parámetros libres...
        self.b_url = params['b_url']
        self.b_title = params['b_title']
        self.b_header = params['b_header']
        self.b_body_hits = params['b_body_hits']
        self.b_anchor = params['b_anchor']
        self.k1 = params['k1']
        self.pagerank_lambda = params['pagerank_lambda']
        self.pagerank_lambda_prime = params['pagerank_lambda_prime']

        # ... y aqui tres estructuras de datos adicionales, necesarias para la implementación
        # (relativas al cálculo de longitudes totales de cada documento, longitudes medias
        # para cada campo, y scorings previos de cada documento por su pagerank):
        self.lengths = {} # Document -> field -> length
        self.avg_length = {}
        self.pagerank_scores = {}

        # Cálculo inicial de las longitudes medias por campo (ver definición de método
        # calc_avg_length() justo a continuación):
        self.calc_avg_length()

    def calc_avg_length(self, debug=False):
        """ Computa las longitudes medias de cada campo en la colección.
        """
        ### BEGIN YOUR CODE (FIXME)
        if(debug):
            print("------ Computando las longitudes medias de cada campo en la colección ...   -------")
        count_docs, count_queries = 0, 0
        count_docs_with_title, cum_len_title = 0, 0
        count_docs_with_headers, cum_len_headers = 0, 0
        count_docs_with_anchors, cum_len_anchors = 0, 0
        count_docs_with_url, cum_len_url = 0, 0
        count_docs_with_body_hits, cum_len_body_hits = 0, 0
        self.lengths = {}
        for i,q in enumerate(query_dict):
            docs = query_dict[q]
            count_queries += 1
            for j,d in enumerate(docs):
                count_docs += 1
                doc = query_dict[q][d]
                self.lengths[doc.url] = {}
                # Campo 'title':
                len_title = 0
                if doc.title:
                    count_docs_with_title += 1
                    len_title = len(doc.title.split())
                    cum_len_title += len_title
                self.lengths[doc.url]["title"] = len_title
                # Campo 'header':
                len_headers = 0
                if doc.headers:
                    count_docs_with_headers += 1
                    for header in doc.headers:
                        len_headers += len(header.split())
                    cum_len_headers += len_headers
                self.lengths[doc.url]["header"] = len_headers
                # Campo 'anchors'
                len_anchors = 0
                if doc.anchors:
                    count_docs_with_anchors += 1
                    for anchor in doc.anchors.keys():
                        len_anchors += len(anchor.split()) * doc.anchors[anchor]
                    cum_len_anchors += len_anchors
                self.lengths[doc.url]["anchors"] = len_anchors
                # Campo 'url'
                len_url = 0
                if doc.url:
                    count_docs_with_url += 1
                    _, dict_counters_url = self.parse_url(doc.url)
                    for c in dict_counters_url.keys():
                        len_url += dict_counters_url[c]
                    cum_len_url += len_url
                self.lengths[doc.url]["url"] = len_url
                # Campo 'body_hits'
                len_body_hits = 0
                if doc.body_length:
                    count_docs_with_body_hits += 1
                    len_body_hits = doc.body_length
                    cum_len_body_hits += len_body_hits
                self.lengths[doc.url]["body_hits"] = len_body_hits
                # Sólo para depurar:
                # if i<=8 and j==4:
                    # print(f"TITLE={doc.title}, LEN={len(doc.title.split())}")
                    # print(f"HEADERS={doc.headers}, LEN={len_headers}")
                    # print(f"ANCHORS={doc.anchors}, LEN={len_anchors}")
                    # print(f"URL={doc.url}={self.parse_url(doc.url)}, LEN={len_url}")
                    # print(f"BODY_LENGTH={doc.body_length}, LEN={len_body_hits}")
        avg_len_title = cum_len_title/count_docs_with_title
        avg_len_headers = cum_len_headers/count_docs_with_headers
        avg_len_anchors = cum_len_anchors/count_docs_with_anchors
        avg_len_url = cum_len_url/count_docs_with_url
        avg_len_body_hits = cum_len_body_hits/count_docs_with_body_hits

        if(debug):
            print(f"count_queries={count_queries}\n"
                f"count_docs={count_docs}\n"
                f"count_docs_with_title={count_docs_with_title}\n"
                f"count_docs_with_headers={count_docs_with_headers}\n"
                f"count_docs_with_anchors={count_docs_with_anchors}\n"
                f"count_docs_with_url={count_docs_with_url}\n"
                f"count_docs_with_body_hits={count_docs_with_body_hits}\n")
            print(f"avg_len_title={avg_len_title}")
            print(f"avg_len_headers={avg_len_headers}")
            print(f"avg_len_anchors={avg_len_anchors}")
            print(f"avg_len_url={avg_len_url}")
            print(f"avg_len_body_hits={avg_len_body_hits}")

        self.avg_length = {"title": avg_len_title, "headers": avg_len_headers,
                           "anchors": avg_len_anchors, "url": avg_len_url,
                           "body_hits": avg_len_body_hits}
        # self.length = {}
        # self.pagerank_scores = {}
        if(debug):
            print("------ ...Completado (longitudes medias de la colección computadas) -------")
        ### END YOUR CODE (FIXME)

    def normalize_doc_vec(self, q, d, doc_vec, debug=False):
        """ Normalizar las frecuencias crudas de los diferentes campos en el documento
            d usando la ecuación (1) especificada más arriba.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            doc_vec (dict) : El vector de documento a normalizar.
        Return:
            doc_vec (dict) : El vector de documento normalizado.
        """
        ### BEGIN YOUR CODE (FIXME)
        if debug:
            print("NORM BEFORE:", doc_vec)
        for key_field in doc_vec.keys():
            if key_field == "url":
                Bf = self.b_url
                lendf = self.lengths[d.url]["url"]
                avlenf = self.avg_length["url"]
            elif key_field == "title":
                Bf = self.b_title
                lendf = self.lengths[d.url]["title"]
                avlenf = self.avg_length["title"]
            elif key_field == "header":
                Bf = self.b_header
                lendf = self.lengths[d.url]["headers"]
                avlenf = self.avg_length["headers"]
            elif key_field == "body_hits":
                Bf = self.b_body_hits
                lendf = self.lengths[d.url]["body_hits"]
                avlenf = self.avg_length["body_hits"]
            elif key_field == "anchor":
                Bf = self.b_anchor
                lendf = self.lengths[d.url]["anchors"]
                avlenf = self.avg_length["anchors"]
            for key_term in doc_vec[key_field].keys():
                doc_vec[key_field][key_term] /= (1+Bf*((lendf/avlenf)-1))
        if debug:
            print("NORM AFTER:", doc_vec)

        return doc_vec
        ### END YOUR CODE (FIXME)

    def get_net_vector(self, q, d):
        """ Obtener el vector neto global para el par (q,d), usando la ecuación (2) anterior.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
        Return:
            doc_vec (dict) : El vector de documento normalizado (ya sólo uno, incluyendo todos los términos incluídos en todos los campos).
        """
        ### BEGIN YOUR CODE (FIXME)
        n_vec = {}
        d_vec = self.get_doc_vector(q, d)
        for field_type in ["url", "title", "headers", "anchors", "body_hits"]:
            if field_type in d_vec.keys():
                for term in d_vec[field_type].keys():
                    if term not in n_vec.keys():
                        n_vec[term] = 0
                    n_vec[term] += d_vec[field_type][term] * self.weights[field_type]
        return n_vec
        ### END YOUR CODE (FIXME)

    def get_net_score(self, q, d):
        """ Obtener la puntuación global BM25F para el par (q,d), usando la ecuación (3) anterior.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
        Return:
            doc_vec (dict) : El scoring neto global, incluyendo ya también la puntuación por pagerank.
        """
        q_vec = self.get_query_vector(q)
        n_vec = self.get_net_vector(q, d)
        score = 0
        for term in n_vec.keys():
            if term in q_vec.keys():
                score += (n_vec[term]/(self.k1+n_vec[term])) * self.idf.get_idf(term)
        score += self.pagerank_lambda * math.log(self.pagerank_lambda_prime + d.pagerank)
        # print("PAGERANK:", d.pagerank)
        return score

Probamos la nueva clase _BM25FScorer_, primero con unos ciertos parámetros iniciales en los que hemos fijado $b_f=0 \quad \forall f$, así como $k_1=\lambda = \lambda'=0$, simplemente para depurar:

In [ ]:
# Imprimimos la consulta y el documento de ejemplo:
q = Query("stanford aoerc pool hours")
d = query_dict[q]['http://events.stanford.edu/2014/February/18/']
print("CONSULTA:", q,"\n")
print("DOCUMENTO:", d)

# Usamos consulta booleana, sin normalizar, e incluyendo en ella el IDF...
query_weight_scheme = {"tf": 'b', "df": 't', "norm": None}
# ... y con conteo de frecuencias crudas iniciales para el documento, normalizados por
# zonas de acuerdo a la ecuación (1):
doc_weight_scheme = {"tf": 'n', "df": 'n', "norm": "default"}

# Creamos el scorer BM25F con los anteriores parámetros, e inicialmente con los respectivos b_f
# inicializados a 0.0 para comprobar la corrección de los cálculos en los vectores separados
# por campos:
params_bm25f = {
    "url_weight" : 0.1,
    "title_weight": 0.15,
    "body_hits_weight" : 0.2,
    "header_weight" : 0.25,
    "anchor_weight" : 0.30,
    "b_url" : 0.0,
    "b_title" : 0.0,
    "b_header" : 0.0,
    "b_body_hits" : 0.0,
    "b_anchor" : 0.0,
    "k1": 0.0,
    "pagerank_lambda" : 0.0,
    "pagerank_lambda_prime" : 0.0,
}
bm25f_scorer = BM25FScorer(theIDF, query_dict, params_bm25f, query_weight_scheme, doc_weight_scheme)

print('\nVector de consulta:', bm25f_scorer.get_query_vector(q))
print('\nVector de documento:', bm25f_scorer.get_doc_vector(q, d))
print('\nVector neto:', bm25f_scorer.get_net_vector(q, d))
print('\nScoring neto:', bm25f_scorer.get_net_score(q, d))

assert bm25f_scorer.get_net_score(q, d)  == sum([theIDF.get_idf(term) if term in q else 0 for term, val in bm25f_scorer.get_net_vector(q, d).items()]), \
       "Fallo en chequeo de la clase BM25FScorer"

CONSULTA: stanford aoerc pool hours 

DOCUMENTO: url: http://events.stanford.edu/2014/February/18/
 title: events at stanford tuesday february 18 2014
 headers: ['stanford university event calendar', 'teaching sex at stanford', 'rodin the complete stanford collection', 'stanford rec trx suspension training', 'memorial church open visiting hours', 'alternative transportation counseling tm 3 hour stanford univ shc employees retirees family members']
 body_hits: {'stanford': [239, 271, 318, 457, 615, 642, 663, 960, 966, 971], 'aoerc': [349, 401, 432, 530, 549, 578, 596], 'pool': [521]}
 body_length: 981
 pagerank: 1


Vector de consulta: Counter({'aoerc': 4.995630807762446, 'pool': 2.446627545736658, 'hours': 1.2882309766291968, 'stanford': 0.14313251558629017})

Vector de documento: {'url': Counter({'events': 1.0, 'stanford': 1.0, 'edu': 1.0, '2014': 1.0, 'february': 1.0, '18': 1.0}), 'title': Counter({'events': 1.0, 'at': 1.0, 'stanford': 1.0, 'tuesday': 1.0, 'february': 1.0, '18': 1.0,

Si observamos la información impresa por la celda anterior, y prestamos atención al vector de documento impreso (separado por campos), comprobamos que los vectores resultantes coinciden con los de conteo originales (como corresponde a los valores $b_f=0 \quad \forall f$ usados para depurar en primera instancia).

Obsérvese también que el vector neto combina ya todos los términos en un sólo vector, según los pesos indicados en los parámetros usados (p.e., para el término _"events"_, que aparece con un valor de 1.0 tanto en el campo **url** como en el campo **title**, el valor es de 0.25, como corresponde a la suma $0.1*1.0+0.15*1.0 = 0.25$, con $w_u=0.1$ y $w_t=0.15$. El término _"aoerc"_, por su parte, aparece con un valor 1.4, correspondiente en este caso a la suma $0.2*7.0 = 1.4$, al aparecer únicamente en el campo **body_hits**, con $w_b=0.2$ y un conteo de apariciones de exactamente 7.0 en dicho campo.

Finalmente, puede comprobarse también que el _scoring_ neto obtenido coincide en este caso con la simple suma de los IDF de todos los términos incluidos en la consulta, como debe ser el caso al aplicar la ecuación (3) con $k_1=\lambda = \lambda'=0$.

Un ejercicio interesante es cambiar ahora los parámetros libres con otros valores con más sentido, para observar sus respectivas influencias en los resultados. Por ejemplo:

In [ ]:
params_bm25f = {
    "url_weight" : 0.1,
    "title_weight": 0.1,
    "body_hits_weight" : 0.1,
    "header_weight" : 0.1,
    "anchor_weight" : 0.1,
    "b_url" : 0.5,
    "b_title" : 0.5,
    "b_header" : 0.5,
    "b_body_hits" : 0.5,
    "b_anchor" : 0.5,
    "k1": 1.0,
    "pagerank_lambda" : 1.0,
    "pagerank_lambda_prime" : 2.0,
}
bm25f_scorer = BM25FScorer(theIDF, query_dict, params_bm25f, query_weight_scheme, doc_weight_scheme)

print('\nVector de consulta:', bm25f_scorer.get_query_vector(q))
print('\nVector de documento:', bm25f_scorer.get_doc_vector(q, d))
print('\nVector neto:', bm25f_scorer.get_net_vector(q, d))
print('\nScoring neto:', bm25f_scorer.get_net_score(q, d))


Vector de consulta: Counter({'aoerc': 4.995630807762446, 'pool': 2.446627545736658, 'hours': 1.2882309766291968, 'stanford': 0.14313251558629017})

Vector de documento: {'url': Counter({'events': 1.1888553231609935, 'stanford': 1.1888553231609935, 'edu': 1.1888553231609935, '2014': 1.1888553231609935, 'february': 1.1888553231609935, '18': 1.1888553231609935}), 'title': Counter({'events': 0.9730647184709617, 'at': 0.9730647184709617, 'stanford': 0.9730647184709617, 'tuesday': 0.9730647184709617, 'february': 0.9730647184709617, '18': 0.9730647184709617, '2014': 0.9730647184709617}), 'headers': Counter({'stanford': 4.865323592354809, 'university': 0.9730647184709617, 'event': 0.9730647184709617, 'calendar': 0.9730647184709617, 'teaching': 0.9730647184709617, 'sex': 0.9730647184709617, 'at': 0.9730647184709617, 'rodin': 0.9730647184709617, 'the': 0.9730647184709617, 'complete': 0.9730647184709617, 'collection': 0.9730647184709617, 'rec': 0.9730647184709617, 'trx': 0.9730647184709617, 'sus

Es interesante reevaluar varias veces la celda anterior jugando con pequeños cambios aislados en los diferentes parámetros libres, e interpretar de esta forma su efecto inmediato tanto en los vectores de documento separados por campos como en el vector neto resultado, y el correspondiente scoring neto final.

# Función de ranking 3: ventana más pequeña

Finalmente, añadiremos la influencia de los tamaños de ventana al algoritmo BM25F. Para una consulta deda $q$, definimos la _ventana más pequeña_ $w_{q,d}$ como la secuencia más corta de tokens en el documento $d$ tal que todos los términos en la consulta $q$ están presentes en dicha secuencia. Una ventana sólo puede especificarse para un campo en particular, y para el caso concreto del campo _anchor\_text_, se exige que todos los términos en $q$ estén presentes en un enlace particular (esto es, si un término ocurre en el texto de un enlace, y otro en el de otro enlace diferente al mismo documento), no se considerará una misma ventana. Si, por otro lado, $d$ no contiene alguno de los términos de la consulta y, por tanto, no se puede encontrar dicha ventana, entonces definimos $w_{q,d} = \infty$.

Intuitivamente, cuanto más pequeña sea la ventana $w_{q,d}$, más relevante debería ser el documento $d$ para la consulta $q$. Por lo tanto, multiplicaremos el _scoring_ BM25F del documento por un factor de _boost_ basado en $w_{q,d}$, de forma que:

* Si $w_{q,d} = \infty$, entonces el factor de _boost_ es 1.0.
* Si $w_{q,d} = |Q|$, siendo $Q$ el número de términos únicos en $q$, entonces multiplicaremos el _scoring_ original por un factor predeterminado máximo $B$, estrictamente mayor que uno.
* Para valores de $w_{q,d}$ entre la longitud de la consulta e infinito, el factor de _boost_ deberá moverse entre B (valor máximo) y 1.0 (valor mínimo), decrementándose progresivamente conforme crece el tamaño de $w_{q,d}$.

Para esto último, podría aquí usarse un decrecimiento de tipo exponencial, o bien del tipo $\frac{1}{x}$. La siguiente gráfica ilustra una posible implementación de este último tipo de decrecimiento, para 4 valores diferentes de B:

In [ ]:
# Ilustramos cuatro funciones de decrecimiento basado en 1/x para los valores
# máximos de B = {4.0, 2.0, 1.75, 1.25}:
len_q_list = 10.0
max_win_len = 50
len_min = np.arange(len_q_list,max_win_len+1)
for B in [4.0, 2.0, 1.75, 1.25]:
    factor_win = 1.0+(B-1.0)*len_q_list/len_min
    plt.plot(len_min,factor_win,'+-');

##  Clase _WindowScorer_

He aquí la definición de una clase _SmallestWindowScorer_ para implementar la técnica anterior. Se basa en la clase anterior _BM25Scorer_, ampliándola fundamentalmente con el método `get_boost_score`, que realiza el trabajo principal apoyándose a su vez en el método `min_sublist_with_all`. Éste último es el que en última instancia realiza la búsqueda efectiva de la ventana de texto más pequeña que contiene a todos los términos de la consulta:

In [ ]:
class SmallestWindowScorer(BM25FScorer):
    def __init__(self, idf, query_dict, params, query_weight_scheme=None, doc_weight_scheme=None):
        super().__init__(idf, query_dict, params, query_weight_scheme=query_weight_scheme, doc_weight_scheme=doc_weight_scheme) #modified
        # Añadimos el parámetro "B" (máximo boosting alcanzable):
        self.B = params["B"]

    ### BEGIN YOUR CODE (FIXME)
    def min_sublist_with_all(self, A, B):
        """ Calcula el tamaño de la mínima sublista de B que contiene a toda la
            lista de términos de A.
        Args:
            A (lista de términos) : Lista de términos en la consulta.
            B (lista de términos) : Lista de términos en la que realizar la
                                    búsqueda de la sublista más pequeña.
        Return:
            min_length (dict) : La longitud de la sublista más pequeña encontrada
                                (float('inf') si no existe tal sublista).
        """
        required_elements = len(A)        # Número de elementos a encontrar
        element_count = defaultdict(int)  # Para contar elementos encontrados
        left = 0                          # Borde izquierdo

        # Convertimos A en diccionario para hacer búsquedas rápidas:
        required = {element: 0 for element in A}

        min_length = float('inf')
        min_start = 0
        elements_found = 0

        # Vamos aumentando el borde derecho:
        for right in range(len(B)):
            if B[right] in required:
                # Encontramos elemento de A en posición 'right' de B.
                element_count[B[right]] += 1
                if element_count[B[right]] == 1:
                    elements_found += 1

            # Intentamos hacer la longitud de la lista lo menor posible...
            while elements_found == required_elements:
                if right - left + 1 < min_length:
                    min_length = right - left + 1
                    min_start = left
                # ... recortando por la izquierda:
                if B[left] in required:
                    element_count[B[left]] -= 1
                    if element_count[B[left]] == 0:
                        elements_found -= 1
                left += 1

        # Devolvemos la longitud de la mínima sublista encontrada:
        return min_length
    ### END YOUR CODE (FIXME)

    def get_boost_score(self, q, d, debug=False):
        """ Calcula el factor de boost basado en la técnica 'smallest window'.
        Args:
            q (Query) : La consulta.
            d (Document) : El documento.
            debug (bool) : Flag para imprimir posible información de depuración.
        Return:
            factor_win (float) : Factor de boost, entre 1 y B.
        """
        ### BEGIN YOUR CODE (FIXME)
        # Lista de términos de la consulta de entrada:
        q_list = str(q).split()

        # Extraemos todas las listas de términos del documento, separadas
        # y procesadas debidamente según campos:
        all_lists = []
        try: # Campo url:
            d_url_list, _ = self.parse_url(d.url)
        except:
            d_url_list = []
        try: # Campo title:
            d_title_list = d.title.split()
        except:
            d_title_list = []
        try: # Campo headers:
            d_headers_lists = [h.split() for h in d.headers]
        except:
            d_headers_lists = []
        try: # Campo anchors:
            d_anchors_lists = [a.split() for a in d.anchors]
        except:
            d_anchors_lists = []
        try: # Campo body:
            # Construimos lista ficticia de términos a partir de body_hits,
            # rellenando con "-" donde no se conoce el término:
            max_pos = -1
            for k in d.body_hits.keys():
                max_pos_k = max(d.body_hits[k])
                if max_pos_k > max_pos:
                    max_pos = max_pos_k
            d_body_hits_list = (max_pos+1)*["-"]
            for k in d.body_hits.keys():
                for pos in d.body_hits[k]:
                    d_body_hits_list[pos] = k
        except:
            d_body_hits_list = []

        # Lista de todas las listas de términos a procesar para este documento:
        all_lists += [d_url_list] + [d_title_list] + d_headers_lists + d_anchors_lists + [d_body_hits_list]
        if debug:
            print(f"\nq_list: {q_list}")
            print("\nall_lists:")
            for l in all_lists:
                print(f" {l}")

        # Cómputo de la mínima ventana para todas las listas de todos los campos
        # (tamaño definitivo de la mínima ventana para este documento):
        if debug:
            print(f"\n Ternas (q_list, lista, min_dist):")
        len_min = float("inf")
        for i,lt in enumerate(all_lists):
            min_dist = self.min_sublist_with_all(q_list,lt)
            if debug:
               print(f"{q_list}     {lt}     {min_dist}")
            if min_dist < len_min:
                len_min = min_dist
        factor_win = 1.0+(self.B-1)*len(q_list)/len_min if len_min != float('inf') else 1.0
        return factor_win
        ### END YOUR CODE (FIXME)

    def get_net_score(self, q, d):
        """ Obtener el scoring neto para un par consulta - documento utilizando
            un factor de boosting computado usando la similaridad por la técnica
            del mínimo tamaño de ventana.
        Args:
            d (Document) : El documento.
            q (Query) : La consulta.

        Return:
            El scoring crudo multiplicado por el factor de boost.
        """
        boost = self.get_boost_score(q, d)
        raw_score = super().get_net_score(q, d)
        return boost * raw_score

Probamos en la celda siguiente la clase anterior. Definimos unos parámetros por defecto para la clase (incluyendo un $B$ máximo de 2.0), elegimos una consulta _q_ y un documento _d_ de prueba, y calculamos un factor de _boost_ en modo `debug=True`, para comprobar la corrección de los cómputos intermedios para calcularlo:

In [ ]:
# Parámetros para la creación de la clase:
params_window = {
    "B": 2.0,
    "url_weight" : 1.0,
    "title_weight": 0.1,
    "body_hits_weight" : 0.25,
    "header_weight" : 0.5,
    "anchor_weight" : 0.3,
    "b_url" : 0.0,
    "b_title" : 0.0,
    "b_header" : 0.0,
    "b_body_hits" : 0.0,
    "b_anchor" : 0.0,
    "k1": 2.0,
    "pagerank_lambda" : 0.1,
    "pagerank_lambda_prime" : 1.0,
}

# Consulta y documento de prueba:
q = Query("stanford parking")
d = query_dict[q]["https://transportation.stanford.edu/"]

# Definición de instancia de la clase:
smallest_window_scorer = SmallestWindowScorer(theIDF, query_dict, params_window)

# Prueba de funcionamiento interno del método get_boost_score(...):
smallest_window_scorer.get_boost_score(q, d, debug=True)

# Prueba del método get_net_score(...) que calcula el scoring neto:
print(f"\nScoring neto tras usar el factor de boost: {smallest_window_scorer.get_net_score(q, d):5.3f}")

# Ranking del dataset

## Clase _Rank_

Definimos una sencilla clase conteniendo sólo métodos de clase, que agrupa varias funciones de utilidad en la construcción de _rankings_ de documentos resultado de la búsqueda para una determinada consulta:

In [ ]:
class Rank:
    # Sólo métodos de clase:
    def score(query_dict, score_type, idf, params):
        """ Llamar a esta función para puntuar (y ordenar según esta puntuación)
            todos los documentos correspondientes a una consulta, en un conjunto
            completo (dado en forma de mapping consultas -> {documentos}).
        Args:
            query_dict (dict) :  Mapeo Query->url->Document.
            score_type (str) : Tipo de scorer a usar ("baseline", "cosine", "bm25f", "window").
            idf (dict) : Diccionario IDF.
            params(dict) : Parámetros para el scorer usado.
        Return
            query_rankings (dict) : Un mapeo Query->Document->(r,s) (r=ranking=entero, comenzando en 1; s=score=float).
        """
        # Seleccionar subclase concreta de AbstractScorer para crear el tipo de instancia
        # concreta de scorer a utilizar:
        if score_type == "baseline": scorer = BaselineScorer(idf)
        elif score_type == "cosine": scorer = CosineSimilarityScorer(idf, query_dict, params)
        elif score_type == "bm25f": scorer = BM25FScorer(idf, query_dict, params)
        elif score_type == "window": scorer = SmallestWindowScorer(idf, query_dict, params)
        else: print('Tipo erróneo de scorer (debe ser "baseline", "cosine", "bm25f" o "window")!')

        # Diccionario donde se almacenará el mapping consultas->rankings devuelto:
        query_rankings = {}
        # Bucle que recorre todas las consultas en el diccionario de entrada:
        for i, query in enumerate(query_dict.keys()):
            q = query
            doc_and_scores = {}
            ### BEGIN YOUR CODE (FIXME)
            # Bucle que recorre todos los urls para cada consulta, obteniendo el score correspondiente:
            for doc in query_dict[query]:
                d = query_dict[q][doc]
                score = scorer.get_net_score(q, d)
                doc_and_scores[d] = score
            # Ordenamos los documentos por sus scorings...
            sorted_doc_scores = sorted([(d,s) for d,s in doc_and_scores.items()],
                                       key=lambda _: _[1], reverse=True)
            # ... y asignamos el mapeo Document->(ranking,score) resultante de todos los documentos
            # a la consulta correspondiente en el mapeo de salida Query->Document->ranking:
            query_rankings[query] = {d:(r+1,s) for r,(d,s) in enumerate(sorted_doc_scores)}
            ### END YOUR CODE (FIXME)

        return query_rankings

    def write_ranking_to_file(query_rankings, ranked_result_filename):
        """ Función que exporta los rankings obtenidos sobre un dataset de
           consultas-documentos a un fichero de texto.
        Args:
            query_rankings (dict) : Un mapeo Query->Document->ranking (ranking=entero, comenzando en 1).
            ranked_result_filename (str): Ruta al archivo de salida.
        """
        with open(ranked_result_filename, "w") as f:
            for query, docs in query_rankings.items():
                f.write("query: "+ query.__str__() + "\n")
                for doc, rank in docs.items():
                    output_info = "  url: " + doc.url + "\n" + \
                                  "    title: " + doc.title + "\n" + \
                                  "    rank:  " + str(rank[0]) + "\n" + \
                                  "    score: " + str(rank[1]) + "\n"
                    f.write(output_info)
        print(f"¡Escritura de archivo {ranked_result_filename} realizada!")

## Generación de archivos de rankings

Usando la clase `Rank` anterior, realizamos las ordenaciones de todas las consultas contenidas en `query_dict` (provenientes del archivo de _training_ inicial). Realizamos cuatro _rankings_ usando las cuatro técnicas desarrolladas (_"baseline"_, _"cosine"_, _"bm25f"_, _"window"_), guardando cada una de ellas en el correspondiente archivo `output/ranked_train_{tecnica}.txt`:

In [ ]:
for method, params in zip(["baseline", "cosine", "bm25f", "window"], [None, params_cosine, params_bm25f, params_window]):
    query_dict = load_train_data(os.path.join(data_dir, "pa3.signal.train"))
    query_rankings = Rank.score(query_dict, method, theIDF, params)
    Rank.write_ranking_to_file(query_rankings, os.path.join("output", "ranked_train_"+method+".txt"))
    print(f"Rankings realizados para {len(query_rankings)} consultas, (usando el {method} scorer)\n")

A título de ejemplo de los resultados obtenidos, a continuación mostramos los _rankings_ realizados por los cuatro métodos para los diez documentos obtenidos para la primera consulta del _dataset_ de _training_:

In [ ]:
!echo RANKING 1ª CONSULTA, BASELINE:
!head -41 output/ranked_train_baseline.txt
!echo

!echo RANKING 1ª CONSULTA, COSINE:
!head -41 output/ranked_train_cosine.txt
!echo

!echo RANKING 1ª CONSULTA, BM25F:
!head -41 output/ranked_train_bm25f.txt
!echo

!echo RANKING 1ª CONSULTA, WINDOW:
!head -41 output/ranked_train_window.txt
!echo

# Evaluación

## Métrica _Normalized Discounted Cumulative Gain_ (NDCG)

Sea cual sea la función de _ranking_ empleada, para evaluar su eficacia necesitaremos una métrica adecuada, basada en algún tipo de medida de relevancia previa disponible para cada documento en el contexto de una consulta.
Una vez dicha métrica esté disponible, la usaremos para evaluar cada función de _ranking_ ${rf}$ con los datos de test ordenados según la misma.

Usaremos en particular la métrica [NDCG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (_Normalized Discounted Cumulative Gain_). Se trata de una métrica en la que, con un valor obtenido siempre positivo y menor o igual que uno, cuanto más se acerque el valor a uno, mejor será la función de ranking con respecto a los valores de relevancia etiquetados iniciales.

Según esta métrica, para una consulta particular, se define el NDGC de una determinada consulta como:

$$
NDCG(q) = \frac{1}{Z} \sum_{m=1}^{p}\frac{2^{R(q,m)}-1}{log_{2}(1+m)}
\tag{4}
$$

(Nótese que en esta métrica utilizamos la 2ª definición de $DCG_p$ de la página correspondiente en wikipedia, que hace que el numerador se anule para una relevancia=0; con ello se pone mayor énfasis en los documentos ubicados en las primeras posiciones).

Aquí $R(q, m)$ es el juicio de relevancia proporcionado manualmente (preetiquetado en el conjunto _training_) dado al documento$m$ para la consulta $q$. $Z$ es simplemente un factor de normalización, consistente en el valor NDCG ideal para esa consulta. Dicho valor ideal (iNDCG) se calcula simplemente ordenando los documentos por orden decreciente de relevancia, y calculando entonces el NDCG correspondiente usando simplemente $Z=1$. Para el caso particular en que $iNDCG=0$ (correspondiente, según la fórmula anterior, a todos los valores de relevancia iguales a cero), se define simplemente $NDCG(q) = 1$. Finalmente, $p$ es el número de documentos totales que son posibles resultados (_matchings_) para la consulta (un máximo de 10 en nuestro caso).

Con dicha definición de NDCG(q) para una determinada consulta, podemos compuar el NDCG para un conjunto de consultas $Q = \{q_1,...,q_m\}$ simplemente tomando la media de los NDCGs obtenidos para todas las consultas individuales $q_i \in Q$.

## Clase _NDCG_

A continuación definimos una clase específica para implementar la métrica NDCG que acabamos de describir:

In [ ]:
class NDCG:
    def get_rel_scores_from_file(self, filename):
        """ Función que obtiene los valores ground-truth de relevancia para un dataset a
            partir de un fichero de texto correspondiente.
        Args:
            filename (str): Ruta al archivo de entrada.
        """
        # Guardaremos las relevancias de cada documento en una variable de instancia
        # self.rel_scores, de tipo mapeo (diccionario) url->float, con url=str:
        self.rel_scores = {}
        query = ""
        with open(filename, 'r') as f:
            for line in f:
                if line.startswith("query"):
                    query = line.split(":")[-1].strip()
                    url_score = {}
                    self.rel_scores[query] = url_score
                else: # line.startswith("url")
                    tokens = line[line.index(":")+1:].strip().split(" ")
                    url = tokens[0]
                    rel = tokens[1]
                    if float(rel) < 0:
                        rel = 0
                    if url_score is not None:
                        url_score[url] = float(rel)

    def calc_ndcg(self, rels):
        """ Calcula el valor NDCG para una ordenación dada, tomando como entrada la
            lista de relevancias de ground-truth de los documentos, ordenada según la
            función de ranking a evaluar. Se aplica entonces la ecuación (4) anterior, que
            tomará un valor más cercano a uno según la ordenación de dicha lista se acerque
            más a la ordenación natural, de mayor a menor, de la lista de relevancias.
        Args:
            filename (str): Lista de relevancias ground-truth (ordenada según la función de
               ranking a evaluar, NO en orden natural de mayor a menor).
        """
        # Cálculo para la ordenación de entrada:
        local_sum = 0
        for i in range(len(rels)):
            rel = rels[i]
            local_sum += (2**rel - 1) / (math.log((i+1) + 1, 2))
        # Repetimos cálculo con las relevancias ya ordenadas de mayor a menor
        # (para calcular el Z de la ecuación(4))
        sorted_rels = sorted(rels, reverse=True)
        sorted_sum = 0
        for i in range(len(sorted_rels)):
            rel = sorted_rels[i]
            sorted_sum += (2**rel - 1) / (math.log((i+1) + 1, 2))
        # Dividimos la suma original por Z, para devolver el NDCG normalizado (cuyo
        # valor será siempre <= 1.0, con 1.0 correspondiente a una ordenación de relevancias
        # perfectamente coherente con la ordenación proporcionada inicialmente por el
        # algoritmo de ranking).
        if (sorted_sum == 0):
            return 0
        else:
            return local_sum/sorted_sum

    def read_ranking_file_and_compute_ndcg(self, ranked_result_filename):
        """ Lee un fichero de rankings (realizado por cualquiera de los métodos de ranking
            desarrollados) y calcula el ndcq para cada consulta (en variable de instancia
            self.query_ndcg, de tipo mapeo consulta->float, con consulta=str). También deja
            en la variable de instancia self.query_docs un mapeo consulta->[Document], donde
            a cada consulta (en formato cadena) se asigna la lista de documentos pertinente.
            Dichas variables calculadas serán empleadas después por el método write_ndcg_result
            para escribir el resultado de la evaluación en un correspondiente archivo de texto.
        Args:
            ranked_result_filename (str): Fichero de entrada.
        """
        self.query_ndcg = {}
        self.query_docs = {}
        cur_q = ""
        cur_rels = []
        with open(ranked_result_filename, 'r') as f:
            for line in f:
                clean_l = line.strip().split(":")
                l_type = clean_l[0].strip()
                l_content = ":".join(clean_l[1:]).strip()
                if l_type == 'query':
                    # Comienzo siguiente query -> calculamos NDCG de la que acabamos
                    # de terminar:
                    if len(cur_rels) > 0:
                        self.query_ndcg[cur_q] = self.calc_ndcg(cur_rels)
                    # Nueva consulta:
                    cur_q = l_content
                    cur_rels = []
                    self.query_docs[cur_q] = []
                elif l_type == 'url':
                    # Nueva URL -> leer y añadir relevancia de la misma a lista cur_rels:
                    doc = Document(l_content)
                    self.query_docs[cur_q].append(doc)
                    if (cur_q in self.rel_scores) and \
                       (doc.url in self.rel_scores[cur_q]):
                        cur_rels.append(self.rel_scores[cur_q][doc.url])
                    else:
                        print("WARNING: No se encuentra el url %s para la consulta %s"%(doc.url, cur_q))
                elif l_type == 'title':
                    doc.title = l_content
                # ignore debug line for now

        # Última consulta:
        if len(cur_rels) > 0:
            self.query_ndcg[cur_q] = self.calc_ndcg(cur_rels)
            # cur_q = l_content
            # cur_rels = []

    def get_avg_ndcg(self):
        """ Calcula el NDCG medio para todo el dataset, calculado simplemente como
            la media del NDCG calculado para cada una de las consultas del mismo.
        """
        sum_ndcg = 0
        for i in self.query_ndcg:
            sum_ndcg += self.query_ndcg[i]
        return sum_ndcg / len(self.query_ndcg)

    def write_ndcg_result(self, ndcg_result_filename):
        """ Escribe los resultados de la métric NDCG (para cada consulta) en un
            archivo de texto de salida.
        Args:
            ndcg_result_filename (str): Fichero de salida.
        """
        with open(ndcg_result_filename, 'w') as f:
            for query in self.query_ndcg:
                f.write("query: " + query + "\n")
                ndcg_score = self.query_ndcg[query]
                f.write("ndcg: " + str(ndcg_score) + "\n")

                for doc in self.query_docs[query]:
                    f.write("  url: " + doc.url + "\n")
                    f.write("    rating: " + str(self.rel_scores[query][doc.url]) + "\n")
                    f.write("    title: " + doc.title + "\n")
                    f.write("    debug:" + "\n")
        print(f"¡Escritura de fichero {ndcg_result_filename} con resultados NDCG de salida realizada!")

## Evaluación de todos los modelos


Finalmente, usamos aquí las clases `Rank` y `NDCG` para evaluar los rankings realizados tanto sobre el conjunto de _training_ (_signal_) como el de validación (_dev_) con los cuatro métodos desarrollados:

In [ ]:
# Definimos unos parámetros base para el método del coseno con cierto sentido:
params_cosine = {
    "url_weight" : 0.5,
    "title_weight": 0.2,
    "body_hits_weight" : 0.1,
    "header_weight" : 0.1,
    "anchor_weight" : 0.1,
    "smoothing_body_length" : 500
}

# Añadimos ahora unos parámetros adicionales relativamente adecuados para el BM25F:
params_bm25f = {
    **params_cosine,
    "b_url" : 0.5,
    "b_title" : 0.5,
    "b_header" : 0.5,
    "b_body_hits" : 0.5,
    "b_anchor" : 0.5,
    "k1": 1.0,
    "pagerank_lambda" : 1.0,
    "pagerank_lambda_prime" : 1.0
}

# Y finalmente añadimos un parámetro adicional necesario para el método de ventana:
params_window = {
    **params_bm25f,
    "B": 2.0
}

#
for dataset_filename in ["pa3.signal.train", "pa3.signal.dev"]:
    query_dict = load_train_data(os.path.join(data_dir, dataset_filename))
    for method, params in zip(["baseline", "cosine", "bm25f", "window"], [None, params_cosine, params_bm25f, params_window]):
        # Realizamos el scoring con el dataset y el método actual, salvando los resultados
        # en un fichero de texto correspondiente:
        query_rankings = Rank.score(query_dict, method, theIDF, params)
        train_or_dev = dataset_filename.split(".")[-1]
        ranked_result_filename = os.path.join("output", f"ranked_{train_or_dev}_{method}.txt")
        Rank.write_ranking_to_file(query_rankings, ranked_result_filename)

        # Creamos instancia de clase NDCG para realizar la métrica, y la usamos para
        # * Cargar en ella las relevancias ground-truth.
        # * Computar la métrica NDCG usando dichas relevancias y el ranking realizado previamente.
        # * Salvar los resultados NDCG en un fichero de texto correspondiente.
        # * Calcular e imprimir el NDCG medio (global) para el dataset y el método actual.
        ndcg = NDCG()
        ndcg.get_rel_scores_from_file(os.path.join(data_dir, dataset_filename.replace("signal","rel")))
        ndcg.read_ranking_file_and_compute_ndcg(ranked_result_filename)
        ndcg_result_filename = ranked_result_filename.replace("ranked", "ndcg")
        ndcg_result_file = os.path.join("output", ndcg_result_filename)
        ndcg.write_ndcg_result(ndcg_result_filename)
        avg_ndcg = ndcg.get_avg_ndcg()
        print(f"NDCG global obtenido para el método {method} sobre el archivo de dataset {dataset_filename}: {avg_ndcg}\n")

Una inspección rápida de los resultados nos muestra que parecen tener bastante sentido, si bien dejaremos su discusión detallada para el último apartado del _notebook_, "[Discusión](#discusion)".

# Aprendizaje automático de pesos

Hasta ahora, los pesos para las diferentes funciones se han especificado manualmente. Sin embargo, en los sistemas reales, a medida que se añaden más y más posibles señales útiles para la clasificación (_features_ tanto textuales como no textuales), dicha especificación manual puede volverse un desafío, al menos para tener una cierta confianza en que dicha especificación se hace de una forma óptima (o al menos cercana a una configuración óptima).

En esta última sección del _notebook_ emplearemos una sencilla técnica de _machine learning_, en concreto la estimación puntual usando regresión lineal, para permitir que los algoritmos de ordenación "aprendan" automáticamente los pesos necesarios para sus respectivas funciones de _ranking_.

## Entrenamiento de un modelo de regresión lineal

Cualquera de los métodos de _ranking_ que hemos desarrollado acaban asociando a cada consulta $q_i$ dada un conjunto de documentos $d_j, \hspace{.2em} \forall j=1\dots 10$, a cada uno de los cuales asocia a su vez un vector $x_{ij}$ de características (_features_) asociadas al correspondiente par consulta-documento. Se producirá también un valor real de _scoring_ correspondiente, al cual denominaremos $y_{i,j}$ (etiqueta asociada al vector $x_{i,j}$).

En la aproximación denominada "puntual" (_pointwise_), la más sencilla en la que podremos aplicar _machine learning_ a la estimación de los pesos, se desecha completamente la estructura grupal de los datos (agrupados de 10 en 10 por las consultas), viéndose simplemente nuestros datos de entrenamiento como una lista plana de pares $\{(x_{i}, y_{i})\}$. El problema de ajuste de pesos en nuestro problema de _ranking_ equivale entonces a aprender una adecuada función $f(x)$ tal que, aplicada a cada vector $(x_{i})$, consiga un valor de _scoring_ que se ajuste lo más posible a $y_{i}$, esto es, $f(x_{i})≈y_{i} , \forall i=1\dots m$, con $m=|\{documents\}|$.

Una vez establecido el problema de aprendizaje planteado (en nuestro caso, un problema de regresión), lo resolveremos usando una aproximación de las más sencillas propuestas para ello, la **regresión lineal**. Esto significa que parametrizaremos $f$ como una simple función lineal que asignará un _scoring_ a cada vector $x$  correspondiente a una consulta-documento como sigue:

$$
f(x) = wx+b
\tag{5}
$$

Aquí el vector de pesos ${w}$ el término de _bias_ $b$ son exactamente los parámetros que necesitarán ser aprendidos, de forma que se minimice la función de pérdida definida así:

$$
\sum_{i=1}^m (f(x_{i})-y_{i})^2
\tag{6}
$$

Es lo que se conoce en la literatura como **aproximación por mínimos cuadrados ordinaria** (_ordinary least squares_).

## Diseño de los vectores de características

Para ilustrar la técnica de aprendizaje de pesos, usaremos como base la clase `CosineSimilarityScore`, que permite extraer un valor individual de similaridad para cada campo `url`, `title`, `header`, `body` y `anchor` del documento. Así, podremos representar cada par consulta($q_i$)-documento($q_j$) como un vector de cinco dimensiones $x_{ij}=(s_u,s_t,s_h,s_b,s_a)$, donde cada componente $s_f$ se corresponde con el _scoring tf-idf_ correspondiente al campo $f$ del documento (esto es, $s_f = q_i \cdot d_{i,f}$, denotando el operador $\cdot$ el producto escalar entre los correspondientes vectores de consulta $q_i$ y el vector de documento $d_i$ para el campo específico $f$.

**Nota importante:** En ningún caso se debe confundir estos vectores $x_{ij}$ con los vectores de consulta y/o documento. Éstos últimos tienen tantas dimensiones como tamaño tiene el vocabulario, y por tanto, un componente numérico diferente por cada término del mismo. Son vectores, por tanto, que se mueven en una dimensión muy alta, si bien son también tremendamente dispersos (_sparse_), esto es, tienen gran cantidad de ceros. No es, por tanto y obviamente, sobre ellos donde se realiza el aprendizaje, sino sobre estos vectores $x_{ij} \in R^5$, mucho más compactos, en los que cada componente no se corresponde con un término ni de la consulta ni del documento, sino precisamente ya con un valor de _scoring_ calculado para cada par, y correspondiente a cada uno de los cinco campos diferentes $f \in \{url,title,header,body,anchor\}$.

Usaremos el esquema de pesado _ddd.qqq = lnb.btn_ (según la notación SMART), tal y como se implementó en la clase base a utilizar `CosineSimilarityScore`. Recordar que esto significa:

1. Para el documento (_lnb_):
 - Frecuencia logarítmica de los términos del documento.
 - No usar IDF en los términos del documento.
 - Normalizar el vector de documento (si bien usando en este caso la pequeña modificación de cálculo de la norma por longitud de documento que se implementó en la función `normalize_doc_vec` de la clase `CosineSimilarityScorer`).

2. Para la consulta (_btn_):
 - Esquema de conteo booleano en los términos de la consulta.
 - Usar IDF en los términos de la consulta.
 - No normalizar el vector de consulta.

## Clase _PointwiseLR_

Definimos la clase _PointwiseLR_ simplemente para agrupar en ella las tres funciones principales para realizar el aprendizaje de la máquina de regresión lineal sobre la predicción "punto a punto". Esto es, considerando cada par consulta-documento como un punto (=vector) independiente, sin tener en cuenta la estructura de documentos resultado agrupados por consulta.

Las funciones principales son:

* `get_feature_vectors_and_relevances`: para obtener los vectores de entrenamiento en formato numpy (tanto los vectores de entrada como la variable de salida a regresionar).
* `train_model`: para realizar el entrenamiento en sí, usando `sklearn`.
* `predict_with_model`: para usar el modelo entrenado para predecir nuevos valores de relevancia a partir de nuevas entradas.

In [ ]:
class PointwiseLR:

    # def __init__(self):
    #    self.model = None

    def get_feature_vectors_and_relevances(signal_file, idf, relevance_file):
        """
        Crear el conjunto de todos los vectores de características correspondientes
        al fichero de entrenamiento (fichero de señal) y del diccionario IDF, con su
        correspondiente vector de relevancias (ground-truth) obtenidas a partir del
        correspondiente archivo de relevancias.

        Args:
            signal_file: Ruta al archivo de señal.
            relevance_file: Ruta al archivo de relevancias.
            idf: Objeto de la clase Idf (ya inicializado con el IDF de todos los términos de la colección)

        Returns:
            feature_vecs: Array numpy de dimension (N, 5), siendo N el número total de pares (consulta, documento)
                          en el archivo de señal.
            relevance_vec: Array numpy de dimensión (N,), siendo N el número total de pares (consulta, documento)
                           en el archivo de relevancias.

        """
        ### BEGIN YOUR CODE (FIXME)
        doc_weight_scheme = {"tf": 'l', "df": 'n', "norm": "default"}
        query_weight_scheme = {"tf": 'b', "df": 't', "norm": None}
        query_dict = load_train_data(signal_file)
        params_cosine = {
            "url_weight" : 0.0,
            "title_weight": 0.0,
            "body_hits_weight" : 0.0,
            "header_weight" : 0.0,
            "anchor_weight" : 0.0,
            "smoothing_body_length" : 500
        }
        cs = CosineSimilarityScorer(idf, query_dict, params_cosine, query_weight_scheme, doc_weight_scheme)
        ndcg = NDCG()
        ndcg.get_rel_scores_from_file(relevance_file)

        feature_vecs = []
        relevance_vecs = []
        for i, q in enumerate(query_dict.keys()):
            ### BEGIN YOUR CODE (FIXME)
            # Bucle que recorre todos los urls para cada consulta, obteniendo el score correspondiente:
            for j, d_url in enumerate(query_dict[q]):
                d = query_dict[q][d_url]
                q_vec = cs.get_query_vector(q)
                d_vec = cs.get_doc_vector(q, d)
                feature_vec = np.array([cs.get_sim_score(q,d,"url"),
                                        cs.get_sim_score(q,d,"title"),
                                        cs.get_sim_score(q,d,"headers"),
                                        cs.get_sim_score(q,d,"anchors"),
                                        cs.get_sim_score(q,d,"body_hits")])
                feature_vecs.append(feature_vec)
                relevance_vecs.append(ndcg.rel_scores[str(q)][d_url])

        feature_vecs = np.array(feature_vecs)
        relevance_vecs = np.array(relevance_vecs)

        ### END YOUR CODE (FIXME)
        return feature_vecs, relevance_vecs

    def train_model(x, y):
        """ Entrena el modelo de regresión lineal usando la clase LinearRegression del paquete sklearn.
        Args:
                x: Array numpy de dimensión (N, 5) con los vectores x_ij (aplanados, para todas los
                   pares consulta-documento (q_i,d_j). Será la variable independiente para la regresión lineal.

                y: Array numpy de dimensión (N,) con los valores de relevancia para cada uno de los vectores x_ij
                   anteriores. Será la variable dependiente para la regresión lineal.

        Returns:
                El modelo entrenado (usable para predecir con el método siguiente, predict_with_model)
        """
        ### BEGIN YOUR CODE (FIXME)
        model = LinearRegression()
        # Entrenamos el modelo con los datos de entrenamiento:
        model.fit(x, y)
        ### END YOUR CODE (FIXME)
        return model

    def predict_with_model(model, x):
        """ Predice _scorings_ finales (netos) para una lista de vectores de entrada utilizando el modelo entrenado.
        Args:
                x: Array numpy de dimensión (N, 5) con los vectores con las características de entrada de cada par
                   consulta-documento en el que predecir el _scoring_ neto.

        Returns:
                y_pred: Array numpy de dimensión (N,) con las relevancias predichas para cada vector correspondiente a
                        un par consulta-documento, basándose en el modelo de regresión lineal entrenado.
        """
        ### BEGIN YOUR CODE (FIXME)
        predictions = model.predict(x)
        ### END YOUR CODE (FIXME)
        return predictions

En la celda siguiente se prueba toda la funcionalidad de la clase anterior:

In [ ]:
# Obtención de vectores y relevancias de entrenamiento a partir de los archivos de señal y de relevancia:
train_feature_vecs, train_relevances = PointwiseLR.get_feature_vectors_and_relevances("pa3-data/pa3.signal.train", theIDF, "pa3-data/pa3.rel.train")
assert train_feature_vecs.shape == (7026, 5), f"¡Error en creación de vectores de training! {train_feature_vecs.shape} != (7026, 5)"

# Entrenamiento:
model = PointwiseLR.train_model(train_feature_vecs, train_relevances)

# Predicción:
dev_feature_vecs, dev_relevances = PointwiseLR.get_feature_vectors_and_relevances("pa3-data/pa3.signal.dev", theIDF, "pa3-data/pa3.rel.dev")
assert dev_feature_vecs.shape == (1187, 5), f"¡Error en creación de vectores de test! {train_feature_vecs.shape} != (1187, 5)"

dev_predicts = PointwiseLR.predict_with_model(model, dev_feature_vecs)
assert dev_predicts.shape == (1187,), f"¡Error en obtención de predicciones! {train_feature_vecs.shape} != (1187,)"

## Evaluación del modelo aprendido

Finalmente, en esta sección, y usando las predicciones del modelo entrenado, computamos su MSE (error mínimo cuadrático), actualizamos los correspondientes parámetros del método de ranking basado en la similaridad del coseno usando dichos parámetros, y volvemos a realizar el ranking tanto sobre el conjunto de aprendizaje como el de test. También obtenemos las respectivas puntuaciones NDCG para ambos, y las comparamos con las del modelo inicial.

In [ ]:
# Imprimimos los coeficientes del modelo obtenido, y su error cuadrático medio (MSE):
print("Parámetros del modelo: ")
print(f"  Coeficientes['url','title','headers','anchors','body_hits']:  {model.coef_}")
print(f"  Término independiente (descartable, no afecta al ranking): {model.intercept_}")
print ("\nMean Squared Error:", mean_squared_error(dev_relevances, dev_predicts))

# Normalizamos pesos para que sumen 1.0:
weights = model.coef_ / np.sum(model.coef_)

# Inicializamos una nueva instancia del CosineSimilarityScorer con los parámetros aprendidos:
params_cosine_trained = {
    "url_weight" : weights[0],
    "title_weight": weights[1],
    "header_weight" : weights[2],
    "anchor_weight" : weights[3],
    "body_hits_weight" : weights[4],
    "smoothing_body_length" : 500
}
print(f"\nParámetros aprendidos para CosineSimilarityScorer: \n  {params_cosine_trained}\n")

# Repetimos el ranking con dicho modelo, en los datasets de training y de test, salvando
# los resultados en los respectivos ficheros ranked_{train|dev}_cosine_Learned_LR.txt,
# y mostrando las respectivas métricas NDCG obtenidas:

for dataset_filename in ["pa3.signal.train", "pa3.signal.dev"]:
    # Lectura dataset:
    query_dict = load_train_data(os.path.join(data_dir, dataset_filename))
    train_or_dev = dataset_filename.split(".")[-1]
    # Ejecutamos función de ranking (método coseno con parámetros entrenados):
    query_rankings = Rank.score(query_dict, "cosine", theIDF, params_cosine_trained)
    # Salvamos resultados:
    ranked_result_filename = os.path.join("output", f"ranked_{train_or_dev}_cosine_Learned_LR.txt")
    Rank.write_ranking_to_file(query_rankings, ranked_result_filename)
    # Medimos NDCG resultante:
    ndcg = NDCG()
    ndcg.get_rel_scores_from_file(os.path.join(data_dir, dataset_filename.replace("signal","rel")))
    ndcg.read_ranking_file_and_compute_ndcg(ranked_result_filename)
    ndcg_result_filename = ranked_result_filename.replace("ranked", "ndcg")
    ndcg_result_file = os.path.join("output", ndcg_result_filename)
    ndcg.write_ndcg_result(ndcg_result_filename)
    avg_ndcg = ndcg.get_avg_ndcg()
    print(f"NDCG global obtenido para el método cosine con parámetros aprendidos mediante LR sobre el archivo de dataset {dataset_filename}: {avg_ndcg}\n")

Ambos NDCGs salen algo mejores que los del modelo del coseno sin entrenar:
* Mejora NDCG _training_: 0.8161916184849138 $→$ 0.8261348918944399
* Mejora NDCG _test_: 0.8016941532045303 $→$ 0.8070937216021422


<a id="discusion"></a>

# Discusión

En todo caso, es obvio que la aproximación de aprendizaje por regresión lineal es un tanto ingenua por varias razones, lo que hace que no se mejoren demasiado significativamente los resultados de base obtenidos por el método del coseno (en los que se empleó un peso homogéneo para cada uno de los cinco campos). Además, el método del coseno, incluso mejorado con los pesos aprendidos, es aún inferior al BM25F (el que a priori mostró los mejores resultados):

* NDCG con BM25F (_train_): 0.8433854652383972
* NDCG con BM25F (_test_): 0.8295600444655024

Es interesante también comentar que, en nuestro caso, el cómputo del _boosting_ por el factor de ventana no mejoró dicho método (lo que no es tan extraño, ya que, si se observan el tipo de la mayoría de consultas realizadas, no se basan tanto en la búsqueda de frases más o menos largas, sino más bien en la utilización de unos pocos términos clave, lo que de alguna manera limita bastante más la utilidad de la técnica):

* NDCG con ventana (_train_): 0.8394620230330289
* NDCG con ventana (_test_): 0.8236233212551707

Algunas conclusiones adicionales del estudio:

1. Tiene cierta lógica que el método BM25F sea el que mejor se comporta. Es al fin y al cabo uno de los más utilizados, y además en nuestro caso es el más completo porque utiliza un tipo de normalización de longitud especializada por campos, añade también la información adicional de la _feature_ no textual _pagerank_, etc., cosas que en nuestra implementación no hemos añadido obviamente al resto de funciones de _ranking_.

2. Una mejora significativa que podría añadirse al algoritmo de aprendizaje sería implementar un algoritmo que **sí** tuviera en cuenta la estructura del conjunto de aprendizaje. Esto es, tener en cuenta que lo que nos interesa es que se nos haga un buen _ranking_ de los resultados **para una consulta dada**, y no tanto que, para todos los pares consulta-documento, se ajuste una misma función de _ranking_, tal y como hemos implementado con nuestro regresor lineal. Se trataría, pues, de implementar un algoritmo de aprendizaje _pairwise_ (en contraposición al esquema _pointwise_ que nosotros hemos utilizado). En dicho enfoque, cada ejemplo consiste en realidad en un par de documentos resultado para una consulta dada, y su etiqueta de ground-truth, si el primero tiene mayor relevancia que el segundo. En ese contexto, cada consulta que posea N posibles resultados genera $N\times(N-1)/2$ pares de ejemplo. Sin embargo, esto exige aplicar algoritmos de aprendizaje algo más avanzados que un regresor lineal, lo que quedaría ya un poco fuera del alcance de los objetivos básicos de este tema. En todo caso, he aquí un enlace a un paquete de python con el que se podría abordar el problema, y donde se introduce un poco este tema del aprendizaje de una función de ranking: [https://xgboost.readthedocs.io/en/stable/tutorials/learning_to_rank.html](https://xgboost.readthedocs.io/en/stable/tutorials/learning_to_rank.html)

3. Obviamente, todos los métodos con parámetros y funciones de apoyo intercambiables, dependen directamente de una buena elección tanto de los primeros como de las segundas. Esto es particularmente cierto en el caso del método BM25F, al que, al añadirle además el posible factor de _boost_ del tamaño de la ventana, acaba teniendo un espacio de posibles configuraciones de dimensiones bastante considerables, que no han sido exploradas en este notebook.


# Ejercicios

En particular, la última de las conclusiones expuestas el apartado anterior nos motiva a proponer el siguiente **ejercicio**:

Tomando como base la clase _WindowScorer_ (la más completa de las desarrolladas, al combinar BM25F con el uso de la _feature_ no textual adicional del tamaño de la ventana más pequeña que contiene a la consulta), intentar mejorar sus resultados. Para ello:

* Se puede realizar una búsqueda manual (intentando ajustar los diferentes parámetros mediante un simple razonamiento de la importancia que creemos que debe tener cada campo, y/o eligiendo los parámetros adicionales con valores típicos que puedan tener sentido). Recordar que el conjunto completo de parámetros que se pueden configurar para la clase _WindowScorer_ es el siguiente:

```
    params_window = {
        "url_weight" : 0.5,
        "title_weight": 0.2,
        "body_hits_weight" : 0.1,
        "header_weight" : 0.1,
        "anchor_weight" : 0.1,
        "smoothing_body_length" : 500
        "b_url" : 0.5,
        "b_title" : 0.5,
        "b_header" : 0.5,
        "b_body_hits" : 0.5,
        "b_anchor" : 0.5,
        "k1": 1.0,
        "pagerank_lambda" : 1.0,
        "pagerank_lambda_prime" : 1.0
        "B": 2.0
    }

```

* Como alternativa, se puede hacer una búsqueda automatizada (aleatoria, o bien relativamente estructurada) del conjunto de parámetros ajustables, aprovechando la facilidad que nos ofrece el código desarrollado para realizar un bucle que pruebe sistemáticamente muchas variaciones de los mismos (siempre manteniendo un número de combinaciones no tan elevado como para hacerlo computacionalmente viable). Esta tipo de aproximación, por cierto, denominada _"Random|Grid Parameter Search"_ es ampliamente utilizada en muchos ámbitos del _machine learning_ en general, y también en el contexto de la Recuperación de Información que nos ocupa en particular.
* También puede realizarse una clase nueva, basada en la _WindowScorer_, donde se redefinan ciertos métodos para cambiar alguna de las funciones auxiliares (p.e., implementando algún esquema SMART diferente, cambiando la función para el _boosting_ adaptativo por tamaño de ventana mínima, o cualquier otra cosa que creáis que pueda tener sentido).
* Puede resultar también muy ilustrativo crear una nueva clase _RandomScorer_, extremadamente sencilla (sólo hay que basarla en la clase raíz adecuada, y redefinir en ella la función de scoring neto para que, simplemente, devuelva un número aleatorio), y evaluar también su rendimiento vía NDGC tanto en el conjunto de training como en el de test. Con ello, obtendremos unos valores de base que nos pueden orientar sobre el valor mínimo de NDGC que puede obtenerse con un método _dummy_, que en este caso simplemente ordenará aleatoriamente los resultados obtenidos para cada búsqueda, poniendo así mejor en contexto los méritos relativos de unos métodos respecto a otros.

Una vez obtenida nuestra solución óptima, es por supuesto interesante reflexionar sobre el conjunto concreto de parámetros / funciones con las que se ha obtenido dicho óptimo, intentando argumentar sobre los valores finales obtenidos, la importancia relativa de los parámetros más influyentes, y su posible razón, en el contexto del _dataset_ que nos ocupa.

Orientativamente, no obstante, el profesorado de la asignatura _"Information Retrieval"_ de la Universidad de Stanford que creó este dataset para la misma, informó (sin publicar su solución particular) que, combinando varias de las opciones propuestas, logró obtener NDCG sólo marginalmente superiores al ~0.829 obtenido por el BM25F para el conjunto de test (_dev_) configurado con los parámetros por defecto indicados más arriba. Por último, téngase en cuenta también, por supuesto, que **en ningún caso se debe optimizar directamente sobre el conjunto de test**, sino que lo correcto es optimizar los parámetros sobre el NDCG obtenido sobre el training, para posteriormente simplemente informar sobre en NDCG sobre el test al configurar el método con los parámetros óptimos así obtenidos.